# AIE425 Intelligent Recommender Systems, Fall Semester 25/26
Assignment 1: Neighborhood CF & Clustering in CF
| Name | ID | Part |
| ------ | -----| -----|
| Youssef Husseiny | 222101943 | Section 3 |
| Habiba Ahmed | 222100471 | Section 2 |
| Seif Amr Abdelhafez | 222102312 | Section 1 |

## Part 1: User-Based Collaborative Filtering Requirements and Questions:


In [55]:
import pandas as pd
import numpy as np

In [56]:
m = pd.read_csv('../../dataset/movies.csv')
r = pd.read_csv('../../dataset/ratings.csv')

In [57]:
m

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
87580,292731,The Monroy Affaire (2022),Drama
87581,292737,Shelter in Solitude (2023),Comedy|Drama
87582,292753,Orca (2023),Drama
87583,292755,The Angry Breed (1968),Drama


In [58]:
r

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
32000199,200948,79702,4.5,1294412589
32000200,200948,79796,1.0,1287216292
32000201,200948,80350,0.5,1294412671
32000202,200948,80463,3.5,1350423800


In [59]:
import os

In [60]:
os.getcwd()

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\section2_neighborhood_cf\\part1_user_based_cf'

In [61]:
import os
import sys

utils_path = os.path.abspath(os.path.join(os.getcwd(), '../..', 'utils'))

In [62]:
if utils_path not in sys.path:
    sys.path.append(utils_path)

In [63]:
print(f"Calculated utils_path: {utils_path}")
print(f"Is utils_path in sys.path: {utils_path in sys.path}")

Calculated utils_path: d:\My_Laptop\University\GU\AI Science\Year 4\Semester 1\Intelligent Recommender System\Assignments\Assignment1\utils
Is utils_path in sys.path: True


In [64]:
os.listdir(utils_path)

['Cosine_Sim.py',
 'MeanCenteredSimilarityRS.py',
 'Pearson_Sim.py',
 '__init__.py',
 '__pycache__']

In [65]:
utils_path

'd:\\My_Laptop\\University\\GU\\AI Science\\Year 4\\Semester 1\\Intelligent Recommender System\\Assignments\\Assignment1\\utils'

In [66]:
from Cosine_Sim import Cosine_similarity_RS
import importlib


In [67]:
r

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858
...,...,...,...,...
32000199,200948,79702,4.5,1294412589
32000200,200948,79796,1.0,1287216292
32000201,200948,80350,0.5,1294412671
32000202,200948,80463,3.5,1350423800


In [68]:
print(f"Original Data: {r['userId'].nunique()} Users, {r['movieId'].nunique()} Movies")

Original Data: 200948 Users, 84432 Movies


In [69]:
def read_ids_from_file(filepath):
    """Reads IDs from a file."""
    if not os.path.exists(filepath):
        print(f"Warning: File not found {filepath}")
        return []
    with open(filepath, 'r') as f:
        content = f.read().replace('\n', ',') 
        return [int(float(x)) for x in content.split(',') if x.strip().replace('.','',1).isdigit()]

In [70]:
target_users_ids = read_ids_from_file('../../Results/target_users.txt')
target_items_ids = read_ids_from_file('../../Results/target_items.txt')

In [71]:
print(f"Loaded Target Users: {target_users_ids}")
print(f"Loaded Target Items: {target_items_ids}")

Loaded Target Users: [79060, 183483, 55243]
Loaded Target Items: [203146, 230591]


In [72]:
top_movies = r['movieId'].value_counts().head(1200).index.tolist()
target_user_movies = r[r['userId'].isin(target_users_ids)]['movieId'].unique().tolist()
movies_to_keep = list(set(top_movies) | set(target_items_ids) | set(target_user_movies))
r_filtered = r[r['movieId'].isin(movies_to_keep)]

In [73]:
top_users = r_filtered['userId'].value_counts().head(110000).index.tolist()
users_to_keep = list(set(top_users) | set(target_users_ids))

r_filtered = r_filtered[r_filtered['userId'].isin(users_to_keep)]

In [74]:
matrix_df = r_filtered.pivot_table(index='userId', columns='movieId', values='rating')
matrix_user_ids = matrix_df.index.tolist()
matrix_movie_ids = matrix_df.columns.tolist()
rating_matrix_np = matrix_df.values

In [75]:
print(f"Users: {r_filtered['userId'].nunique()} (criteria: >= 100,000)")
print(f"Products: {r_filtered['movieId'].nunique()} (criteria: >= 1,000)")
print(f"Ratings: {len(r_filtered)} (criteria: >= 1,000,000)")

Users: 110003 (criteria: >= 100,000)
Products: 1216 (criteria: >= 1,000)
Ratings: 17728444 (criteria: >= 1,000,000)


In [ ]:
matrix_df = r_filtered.pivot_table(index='userId', columns='movieId', values='rating')
matrix_df

movieId,1,2,3,5,6,7,10,11,16,17,...,185029,187593,192803,195159,202429,202439,203146,204698,207313,208703
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3.5,NaN,NaN,NaN,NaN,4.0,4.0,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,2.5,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,5.0,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200943,3.5,2.5,NaN,NaN,3.5,4.5,3.5,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200944,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
matrix_user_ids = matrix_df.index.tolist()    
matrix_movie_ids = matrix_df.columns.tolist() 
rating_matrix_np = matrix_df.values
rating_matrix_np

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, 3.5, nan, ..., nan, nan, nan],
       [2.5, 2. , nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [4. , nan, nan, ..., nan, nan, nan],
       [5. , 1.5, nan, ..., nan, nan, nan]])

In [ ]:
rating_matrix_np.shape

(110003, 1216)

### Case study 1:


#### 1- Apply a user-based CF method using raw Cosine Similarity to calculate the similarity between each target user and other users.


In [25]:
# i have wrote the code of the cosine similarity from scratch in a previous lab task so i added it into utils and imported it here after use ai to modify it for the current data in this task
rs = Cosine_similarity_RS(rating_matrix_np)
rs.fit()

Model initialized for 110003 users. Ready for lazy similarity calculation.


In [26]:
def get_cosine(uid):
    
    if uid not in matrix_user_ids:
        print(f"User ID {uid} not found in the rating matrix.")
        return

    u_idx = matrix_user_ids.index(uid)
    
    all_sims = rs.compute_similarities(u_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    print(f"\nTarget User: {uid}")
    print(f"{'Rank':<5} | {'Other User ID':<15} | {'Cosine Similarity':<20}")
    
    count = 0
    for idx in sorted_indices:
        sim_score = all_sims[idx]
        other_user_id = matrix_user_ids[idx]
        
        if sim_score <= 0: continue
            
        count += 1
        print(f"{count:<5} | {other_user_id:<15} | {sim_score:.6f}")
        
        if count >= 10: break 
        

In [27]:
get_cosine(target_users_ids[0])


Target User: 79060
Rank  | Other User ID   | Cosine Similarity   
1     | 40325           | 1.000000
2     | 193016          | 1.000000
3     | 199694          | 1.000000
4     | 41349           | 1.000000
5     | 124456          | 1.000000
6     | 103832          | 1.000000
7     | 58407           | 1.000000
8     | 151861          | 1.000000
9     | 131650          | 1.000000
10    | 277             | 1.000000


In [28]:
get_cosine(target_users_ids[1])


Target User: 183483
Rank  | Other User ID   | Cosine Similarity   
1     | 134345          | 1.000000
2     | 46073           | 1.000000
3     | 199627          | 1.000000
4     | 174575          | 1.000000
5     | 90926           | 1.000000
6     | 194139          | 1.000000
7     | 101547          | 1.000000
8     | 31624           | 1.000000
9     | 106800          | 1.000000
10    | 68804           | 1.000000


In [29]:
get_cosine(target_users_ids[2])


Target User: 55243
Rank  | Other User ID   | Cosine Similarity   
1     | 130341          | 1.000000
2     | 176692          | 1.000000
3     | 200679          | 1.000000
4     | 167535          | 1.000000
5     | 104687          | 1.000000
6     | 174421          | 1.000000
7     | 121301          | 1.000000
8     | 151197          | 1.000000
9     | 179068          | 1.000000
10    | 167608          | 1.000000


#### 2- Based on the similarity scores, identify the top 20% most similar users to each target user. 


In [30]:
target_users_ids

[79060, 183483, 55243]

In [31]:
uid1=target_users_ids[0]
uid1

79060

In [32]:
def print_10neighbors(uid):
    u_idx = matrix_user_ids.index(uid)
    all_sims = rs.compute_similarities(u_idx)

    sorted_indices = np.argsort(all_sims)[::-1]

    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)

    top_20_indices = sorted_indices[:top_20_count]


    top_20_users = []
    for idx in top_20_indices:
        user_id = matrix_user_ids[idx]
        score = all_sims[idx]
        if score > 0:
            top_20_users.append((user_id, score))
            
    print(f"\nTarget User: {uid}")

    print(f"\n{'show Top 10 from the Top 20% ':^40}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Similarity':<10}")
    print("-" * 40)

    for rank, (other_uid, score) in enumerate(top_20_users[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.5f}")
    
    return top_20_users

In [33]:
top_20_users_for_u1 = print_10neighbors(target_users_ids[0])


Target User: 79060

     show Top 10 from the Top 20%       
Rank  | User ID         | Similarity
----------------------------------------
1     | 40325           | 1.00000
2     | 193016          | 1.00000
3     | 199694          | 1.00000
4     | 41349           | 1.00000
5     | 124456          | 1.00000
6     | 103832          | 1.00000
7     | 58407           | 1.00000
8     | 151861          | 1.00000
9     | 131650          | 1.00000
10    | 277             | 1.00000


In [34]:
top_20_users_for_u2 = print_10neighbors(target_users_ids[1])


Target User: 183483

     show Top 10 from the Top 20%       
Rank  | User ID         | Similarity
----------------------------------------
1     | 134345          | 1.00000
2     | 46073           | 1.00000
3     | 199627          | 1.00000
4     | 174575          | 1.00000
5     | 90926           | 1.00000
6     | 194139          | 1.00000
7     | 101547          | 1.00000
8     | 31624           | 1.00000
9     | 106800          | 1.00000
10    | 68804           | 1.00000


In [35]:
top_20_users_for_u3 = print_10neighbors(target_users_ids[2])


Target User: 55243

     show Top 10 from the Top 20%       
Rank  | User ID         | Similarity
----------------------------------------
1     | 130341          | 1.00000
2     | 176692          | 1.00000
3     | 200679          | 1.00000
4     | 167535          | 1.00000
5     | 104687          | 1.00000
6     | 174421          | 1.00000
7     | 121301          | 1.00000
8     | 151197          | 1.00000
9     | 179068          | 1.00000
10    | 167608          | 1.00000


#### 3- Use these top similar users to predict unknown ratings for each target user and determine their potential preference for unrated items.


In [ ]:

def generate_recommendations(uid, K=10):
    if uid not in matrix_user_ids:
        print(f"User {uid} not in the matrix.")
        return
        
    u_idx = matrix_user_ids.index(uid)

    user_ratings = rs.matrix[u_idx]
    
    unrated_item_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items (in filtered set): {len(unrated_item_indices)}")
    print("Calculating predictions...")
    
    recommendations = []
    
    for i_idx in unrated_item_indices:
        try:
            pred_rating = rs.pred(u_idx, i_idx, K)
            
            if pred_rating > 0:
                item_id = matrix_movie_ids[i_idx]
                recommendations.append((item_id, pred_rating))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations could be generated (no neighbors rated the unrated items).")
    else:
        print(f"\n  Top 5 Recommended Movies for User {uid}:")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Predicted Rating':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, rating) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {rating:.4f}")
    
    return recommendations

In [37]:
K = 10
recommendations_for_u1 = generate_recommendations(target_users_ids[0], K)


Target User: 79060
Total Unrated Items (in filtered set): 1196
Calculating predictions... (this might take a moment)

  Top 5 Recommended Movies for User 79060:
  Rank  | Movie ID     | Predicted Rating
  ----------------------------------------
  1     | 1136         | 4.8500
  2     | 318          | 4.8000
  3     | 720          | 4.7500
  4     | 1278         | 4.7500
  5     | 104879       | 4.6500


In [38]:
recommendations_for_u2 = generate_recommendations(target_users_ids[1], K)


Target User: 183483
Total Unrated Items (in filtered set): 1194
Calculating predictions... (this might take a moment)

  Top 5 Recommended Movies for User 183483:
  Rank  | Movie ID     | Predicted Rating
  ----------------------------------------
  1     | 171763       | 4.6500
  2     | 912          | 4.5500
  3     | 1537         | 4.5500
  4     | 91542        | 4.5500
  5     | 318          | 4.5000


In [39]:
recommendations_for_u3= generate_recommendations(target_users_ids[1], K)


Target User: 183483
Total Unrated Items (in filtered set): 1194
Calculating predictions... (this might take a moment)

  Top 5 Recommended Movies for User 183483:
  Rank  | Movie ID     | Predicted Rating
  ----------------------------------------
  1     | 171763       | 4.6500
  2     | 912          | 4.5500
  3     | 1537         | 4.5500
  4     | 91542        | 4.5500
  5     | 318          | 4.5000


#### 4- Calculate the Discount Factor (DF) and then the Discounted Similarity (DS) for each target user using threshold ß≥30%.


In [40]:
from Cosine_Sim import Discounted_Cosine_similarity_RS

In [41]:
Drs = Discounted_Cosine_similarity_RS(rating_matrix_np)

In [42]:
Drs.fit()

Model initialized for 110003 users. Ready for lazy similarity calculation.


In [43]:
total_items = rating_matrix_np.shape[1] 
beta_percentage = 0.30
beta_threshold = int(np.ceil(total_items * beta_percentage))

In [44]:
def print_discounted_neighbors(uid):
    if uid not in matrix_user_ids:
        print(f"User {uid} not in the matrix.")
        return        
    u_idx = matrix_user_ids.index(uid)
    raw_sims, counts, dfs, dses = Drs.get_discounted_stats(u_idx, beta_threshold)
    
    sorted_indices = np.argsort(raw_sims)[::-1]
    
    print(f"\nTarget User: {uid}")
    print(f"{'Rank':<4} | {'Neighbor ID':<12} | {'Common':<6} | {'DF':<6} | {'Raw Sim':<10} | {'-->':<3} | {'Disc Sim (DS)':<10}")
    print("-" * 80)
    
    count = 0
    for idx in sorted_indices:
        if raw_sims[idx] <= 0: continue
        
        neighbor_id = matrix_user_ids[idx]
        
        n_raw = raw_sims[idx]
        n_count = int(counts[idx])
        n_df = dfs[idx]
        n_ds = dses[idx]
        
        print(f"{count+1:<4} | {neighbor_id:<12} | {n_count:<6} | {n_df:.4f} | {n_raw:.6f} | --> | {n_ds:.6f}")
        
        count += 1
        if count >= 10: break 
        
    if count == 0:
        print("  No neighbors found.")

In [45]:
print_discounted_neighbors(target_users_ids[0])


Target User: 79060
Rank | Neighbor ID  | Common | DF     | Raw Sim    | --> | Disc Sim (DS)
--------------------------------------------------------------------------------
1    | 40325        | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 193016       | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 199694       | 4      | 0.0110 | 1.000000 | --> | 0.010959
4    | 41349        | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 124456       | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 103832       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 58407        | 3      | 0.0082 | 1.000000 | --> | 0.008219
8    | 151861       | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 131650       | 2      | 0.0055 | 1.000000 | --> | 0.005479
10   | 277          | 3      | 0.0082 | 1.000000 | --> | 0.008219


In [46]:
print_discounted_neighbors(target_users_ids[1])


Target User: 183483
Rank | Neighbor ID  | Common | DF     | Raw Sim    | --> | Disc Sim (DS)
--------------------------------------------------------------------------------
1    | 134345       | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 46073        | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 199627       | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 174575       | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 90926        | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 194139       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 101547       | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 31624        | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 106800       | 3      | 0.0082 | 1.000000 | --> | 0.008219
10   | 68804        | 3      | 0.0082 | 1.000000 | --> | 0.008219


In [47]:
print_discounted_neighbors(target_users_ids[2])


Target User: 55243
Rank | Neighbor ID  | Common | DF     | Raw Sim    | --> | Disc Sim (DS)
--------------------------------------------------------------------------------
1    | 130341       | 3      | 0.0082 | 1.000000 | --> | 0.008219
2    | 176692       | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 200679       | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 167535       | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 104687       | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 174421       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 121301       | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 151197       | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 179068       | 2      | 0.0055 | 1.000000 | --> | 0.005479
10   | 167608       | 2      | 0.0055 | 1.000000 | --> | 0.005479


#### 5- Using the DS values, determine a new list of the top 20% most similar users.


In [48]:
def get_top20_discounted_users(uid):
    if uid not in matrix_user_ids:
        print(f"User {uid} not in the matrix.")
        return
        
    u_idx = matrix_user_ids.index(uid)
    _, _, _, dses = Drs.get_discounted_stats(u_idx, beta_threshold)
    
    sorted_indices_ds = np.argsort(dses)[::-1]
    
    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)
    
    top_20_indices_ds = sorted_indices_ds[:top_20_count]
    
    top_20_users_ds = []
    for idx in top_20_indices_ds:
        user_id = matrix_user_ids[idx]
        score_ds = dses[idx]
        if score_ds > 0:
            top_20_users_ds.append((user_id, score_ds))
            
    print(f"\nTarget User: {uid}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Users with Positive DS in Top 20%: {len(top_20_users_ds)}")
    
    print(f"\n{'- Preview of Top 10 (Based on DS) -':^50}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Discounted Sim':<15}")
    print("-" * 50)
    
    for rank, (other_uid, score) in enumerate(top_20_users_ds[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.6f}")
        
    print("-" * 50)
    return top_20_users_ds

In [49]:
top_20_users_ds_for_u1=get_top20_discounted_users(target_users_ids[0])


Target User: 79060
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

       - Preview of Top 10 (Based on DS) -        
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 55653           | 0.053041
2     | 83000           | 0.052959
3     | 175325          | 0.052809
4     | 9643            | 0.050955
5     | 198515          | 0.050939
6     | 3614            | 0.050760
7     | 129125          | 0.050719
8     | 158697          | 0.050412
9     | 137752          | 0.050284
10    | 17831           | 0.050219
--------------------------------------------------


In [50]:
top_20_users_ds_for_u2 = get_top20_discounted_users(target_users_ids[1])


Target User: 183483
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

       - Preview of Top 10 (Based on DS) -        
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 198515          | 0.053694
2     | 123465          | 0.052727
3     | 175325          | 0.051512
4     | 43703           | 0.050606
5     | 141385          | 0.048878
6     | 131785          | 0.048801
7     | 174601          | 0.048728
8     | 74000           | 0.048724
9     | 10202           | 0.048586
10    | 65006           | 0.048528
--------------------------------------------------


In [51]:
top_20_users_ds_for_u3 = get_top20_discounted_users(target_users_ids[2])


Target User: 55243
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

       - Preview of Top 10 (Based on DS) -        
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 115589          | 0.067889
2     | 33616           | 0.067817
3     | 169154          | 0.067790
4     | 177834          | 0.067783
5     | 134711          | 0.067779
6     | 17700           | 0.067759
7     | 99818           | 0.067752
8     | 139532          | 0.067737
9     | 132183          | 0.067727
10    | 11724           | 0.067723
--------------------------------------------------


#### 6- Again, use this updated similarity to predict missing ratings for each target user.


In [ ]:
def get_discounted_neighbors(K,uid):        
        u_idx = matrix_user_ids.index(uid)
        
        # 1. Identify Unrated Items
        user_ratings = Drs.matrix[u_idx]
        unrated_indices = np.where(np.isnan(user_ratings))[0]
        
        print(f"\nTarget User: {uid}")
        print(f"Total Unrated Items: {len(unrated_indices)}")
        print("Calculating predictions using DS...")
        
        recommendations = []
        
        # 2. Predict for unrated items
        for i_idx in unrated_indices:
            try:
                # Use pred_discounted instead of standard pred
                pred_val = Drs.pred_discounted(u_idx, i_idx, K, beta_threshold)
                
                if pred_val > 0:
                    item_id = matrix_movie_ids[i_idx]
                    recommendations.append((item_id, pred_val))
            except Exception:
                continue
                
        # 3. Sort by Prediction (Descending)
        recommendations.sort(key=lambda x: x[1], reverse=True)
        
        # 4. Display Top 5 Recommendations
        if not recommendations:
            print("  No recommendations generated.")
        else:
            print(f"\n  Top 5 Recommended Movies (DS-Based):")
            print(f"  {'Rank':<5} | {'Movie ID':<12} | {'DS Prediction':<15}")
            print("  " + "-" * 40)
            
            for rank, (movie_id, val) in enumerate(recommendations[:5]):
                print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
                
        print("-" * 60)
        return recommendations

In [53]:
K = 10
discounted_recommendations_for_u1 = get_discounted_neighbors(K,target_users_ids[0])


Target User: 79060
Total Unrated Items: 1196
Calculating predictions using DS... (this might take a moment)

  Top 5 Recommended Movies (DS-Based):
  Rank  | Movie ID     | DS Prediction  
  ----------------------------------------
  1     | 923          | 5.0000
  2     | 1945         | 4.8968
  3     | 858          | 4.8453
  4     | 912          | 4.8453
  5     | 2028         | 4.7957
------------------------------------------------------------


In [54]:
discounted_recommendations_for_u2 = get_discounted_neighbors(K,target_users_ids[1])


Target User: 183483
Total Unrated Items: 1194
Calculating predictions using DS... (this might take a moment)

  Top 5 Recommended Movies (DS-Based):
  Rank  | Movie ID     | DS Prediction  
  ----------------------------------------
  1     | 923          | 4.7061
  2     | 919          | 4.6537
  3     | 858          | 4.6045
  4     | 1213         | 4.5997
  5     | 1203         | 4.5556
------------------------------------------------------------


In [55]:
discounted_recommendations_for_u3 = get_discounted_neighbors(K,target_users_ids[2])


Target User: 55243
Total Unrated Items: 1191
Calculating predictions using DS... (this might take a moment)

  Top 5 Recommended Movies (DS-Based):
  Rank  | Movie ID     | DS Prediction  
  ----------------------------------------
  1     | 1214         | 4.7000
  2     | 2019         | 4.7000
  3     | 858          | 4.6999
  4     | 1258         | 4.6501
  5     | 1203         | 4.6499
------------------------------------------------------------


#### 7- Compare the lists of top users from steps 2 and 5. Discuss any differences or patterns observed.


In [56]:
top_20_users_for_u1_df = pd.DataFrame(top_20_users_for_u1, columns=['UserID', 'CosineSimilarity'])
top_20_users_for_u1_df

,UserID,CosineSimilarity
0,40325,1.000000
1,193016,1.000000
2,199694,1.000000
3,41349,1.000000
4,124456,1.000000
...,...,...
21995,9823,0.995579
21996,149486,0.995579
21997,92920,0.995579
21998,21571,0.995579


In [57]:
top_20_users_ds_for_u1_df = pd.DataFrame(top_20_users_ds_for_u1, columns=['UserID', 'DiscountedSimilarity'])
top_20_users_ds_for_u1_df

,UserID,DiscountedSimilarity
0,55653,0.053041
1,83000,0.052959
2,175325,0.052809
3,9643,0.050955
4,198515,0.050939
...,...,...
21995,195672,0.018086
21996,80630,0.018086
21997,70708,0.018085
21998,84720,0.018085


In [58]:
top20_users_for_u1_compare = top_20_users_for_u1_df.merge(top_20_users_ds_for_u1_df, on='UserID', how='inner')
top20_users_for_u1_compare

,UserID,CosineSimilarity,DiscountedSimilarity
0,179466,0.997265,0.019126
1,74598,0.997156,0.024587
2,103749,0.997109,0.019123
3,143765,0.996874,0.019118
4,99150,0.996841,0.019117
5,160192,0.996836,0.021848
6,150087,0.996730,0.024577
7,33780,0.996699,0.019115
8,133324,0.996546,0.021842
9,185015,0.996437,0.021840


In [59]:
top_20_users_for_u2_df = pd.DataFrame(top_20_users_for_u2, columns=['UserID', 'CosineSimilarity'])
top_20_users_for_u2_df

,UserID,CosineSimilarity
0,134345,1.000000
1,46073,1.000000
2,199627,1.000000
3,174575,1.000000
4,90926,1.000000
...,...,...
21995,160954,0.998221
21996,106362,0.998221
21997,136795,0.998221
21998,41564,0.998221


In [60]:
top_20_users_ds_for_u2_df = pd.DataFrame(top_20_users_ds_for_u2, columns=['UserID', 'DiscountedSimilarity'])
top_20_users_ds_for_u2_df

,UserID,DiscountedSimilarity
0,198515,0.053694
1,123465,0.052727
2,175325,0.051512
3,43703,0.050606
4,141385,0.048878
...,...,...
21995,177042,0.010606
21996,78949,0.010606
21997,167089,0.010606
21998,118655,0.010606


In [61]:
top20_users_for_u2_compare = top_20_users_for_u2_df.merge(top_20_users_ds_for_u2_df, on='UserID', how='inner')
top20_users_for_u2_compare

,UserID,CosineSimilarity,DiscountedSimilarity
0,2098,1.000000,0.016438
1,14165,1.000000,0.013699
2,60687,1.000000,0.016438
3,131971,1.000000,0.013699
4,104535,1.000000,0.010959
...,...,...,...
1439,160954,0.998221,0.010939
1440,106362,0.998221,0.010939
1441,136795,0.998221,0.010939
1442,41564,0.998221,0.010939


In [62]:
top_20_users_for_u3_df = pd.DataFrame(top_20_users_for_u3, columns=['UserID', 'CosineSimilarity'])
top_20_users_for_u3_df

,UserID,CosineSimilarity
0,130341,1.000000
1,176692,1.000000
2,200679,1.000000
3,167535,1.000000
4,104687,1.000000
...,...,...
21995,103059,0.990534
21996,7654,0.990534
21997,164271,0.990534
21998,154699,0.990533


In [63]:
top_20_users_ds_for_u3_df = pd.DataFrame(top_20_users_ds_for_u3, columns=['UserID', 'DiscountedSimilarity'])
top_20_users_ds_for_u3_df

,UserID,DiscountedSimilarity
0,115589,0.067889
1,33616,0.067817
2,169154,0.067790
3,177834,0.067783
4,134711,0.067779
...,...,...
21995,173349,0.037603
21996,32141,0.037602
21997,50950,0.037602
21998,124399,0.037602


In [64]:
top20_users_for_u3_compare = top_20_users_for_u3_df.merge(top_20_users_ds_for_u3_df, on='UserID', how='inner')
top20_users_for_u3_compare

,UserID,CosineSimilarity,DiscountedSimilarity
0,122456,0.997143,0.038247
1,44687,0.997084,0.038244
2,42813,0.997074,0.038244
3,5498,0.996763,0.043694
4,31814,0.996708,0.043691
...,...,...,...
1245,174443,0.990535,0.048848
1246,155043,0.990535,0.040707
1247,103059,0.990534,0.048848
1248,7654,0.990534,0.046134


##### Dissuction
it's very obvious to see that all the cosine similarty decreased, before applying the discounting factor the cosince similarity was nearly perfect-close to 1-, and i suspect that because the high bias of the raw cosine similarity which make it overestimate the similarity, but after applying the discounting factor, there is a clear results now of the similarity between them

#### 8- Compare the rating predictions from steps 3 and 6. Provide comments on the impact of using DS.


In [65]:
recommendations_for_u1

[(1136, 4.85),
 (318, 4.8),
 (720, 4.75),
 (1278, 4.75),
 (104879, 4.65),
 (5618, 4.6499999999999995),
 (1193, 4.6),
 (1244, 4.6),
 (177593, 4.6),
 (1203, 4.55),
 (1269, 4.55),
 (3000, 4.55),
 (7153, 4.55),
 (27773, 4.55),
 (215, 4.5),
 (527, 4.5),
 (1387, 4.5),
 (1957, 4.5),
 (2324, 4.5),
 (2336, 4.5),
 (142488, 4.5),
 (176371, 4.5),
 (1201, 4.45),
 (1230, 4.45),
 (2019, 4.45),
 (5971, 4.45),
 (6016, 4.45),
 (49278, 4.45),
 (96821, 4.45),
 (168248, 4.45),
 (29, 4.4),
 (858, 4.4),
 (908, 4.4),
 (919, 4.4),
 (1172, 4.4),
 (1207, 4.4),
 (1234, 4.4),
 (1732, 4.4),
 (3911, 4.4),
 (7090, 4.4),
 (913, 4.35),
 (1036, 4.35),
 (1210, 4.35),
 (1212, 4.35),
 (1222, 4.35),
 (1357, 4.35),
 (2502, 4.35),
 (26662, 4.35),
 (30749, 4.35),
 (33794, 4.35),
 (77455, 4.35),
 (88810, 4.35),
 (104374, 4.35),
 (168252, 4.35),
 (202439, 4.35),
 (912, 4.3),
 (1077, 4.3),
 (1204, 4.3),
 (1221, 4.3),
 (1252, 4.3),
 (1262, 4.3),
 (1270, 4.3),
 (1272, 4.3),
 (1274, 4.3),
 (1358, 4.3),
 (2366, 4.3),
 (2762, 4.3),
 (

In [66]:
recommendations_for_u1_df = pd.DataFrame(recommendations_for_u1, columns=['MovieID', 'PredictedRating'])
recommendations_for_u1_df

,MovieID,PredictedRating
0,1136,4.85
1,318,4.80
2,720,4.75
3,1278,4.75
4,104879,4.65
...,...,...
1191,63992,2.00
1192,3977,1.95
1193,1499,1.90
1194,3273,1.90


In [67]:
discounted_recommendations_for_u1

[(923, 5.0),
 (1945, 4.896786407890225),
 (858, 4.8453256104353954),
 (912, 4.8453256104353954),
 (2028, 4.795686881731834),
 (1228, 4.795526171067735),
 (1204, 4.7937041023164655),
 (2019, 4.743773376412789),
 (919, 4.69493589725103),
 (898, 4.647613686179142),
 (1221, 4.645998263359079),
 (1207, 4.597613921500443),
 (1304, 4.596552282148126),
 (1252, 4.595577993114617),
 (3435, 4.587168778200976),
 (1172, 4.555875430761488),
 (44555, 4.5466034175266525),
 (1203, 4.546593612682368),
 (1212, 4.546287806088643),
 (1213, 4.545842271022786),
 (1263, 4.544672383268581),
 (750, 4.543971288456971),
 (593, 4.541922865773595),
 (296, 4.496203542319225),
 (48516, 4.495132086712735),
 (913, 4.493348394226032),
 (6016, 4.448564112273508),
 (48394, 4.447321358946605),
 (953, 4.4472604325592595),
 (1193, 4.444816018795329),
 (1084, 4.444516390932288),
 (1231, 4.444379582520007),
 (1617, 4.442694322181018),
 (1266, 4.402467171032098),
 (63082, 4.402013622964869),
 (5995, 4.401103089937549),
 (1293, 

In [68]:
discounted_recommendations_for_u1_df = pd.DataFrame(discounted_recommendations_for_u1, columns=['MovieID', 'DiscountedPredictedRating'])
discounted_recommendations_for_u1_df

,MovieID,DiscountedPredictedRating
0,923,5.000000
1,1945,4.896786
2,858,4.845326
3,912,4.845326
4,2028,4.795687
...,...,...
1191,315,2.005221
1192,1562,1.958749
1193,19,1.952263
1194,173,1.446132


In [69]:
discounted_recommendations_for_u1_compaer = discounted_recommendations_for_u1_df.merge(recommendations_for_u1_df, on='MovieID', how='inner')
discounted_recommendations_for_u1_compaer

,MovieID,DiscountedPredictedRating,PredictedRating
0,923,5.000000,3.50
1,1945,4.896786,3.65
2,858,4.845326,4.40
3,912,4.845326,4.30
4,2028,4.795687,3.95
...,...,...,...
1191,315,2.005221,3.20
1192,1562,1.958749,2.25
1193,19,1.952263,2.55
1194,173,1.446132,2.75


In [70]:
recommendations_for_u2_df = pd.DataFrame(recommendations_for_u2, columns=['MovieID', 'PredictedRating'])
recommendations_for_u2_df

,MovieID,PredictedRating
0,171763,4.65
1,912,4.55
2,1537,4.55
3,91542,4.55
4,318,4.50
...,...,...
1189,4015,2.10
1190,455,2.05
1191,673,1.95
1192,1499,1.65


In [71]:
discounted_recommendations_for_u2_df = pd.DataFrame(discounted_recommendations_for_u2, columns=['MovieID', 'DiscountedPredictedRating'])
discounted_recommendations_for_u2_df

,MovieID,DiscountedPredictedRating
0,923,4.706073
1,919,4.653737
2,858,4.604465
3,1213,4.599699
4,1203,4.555620
...,...,...
1189,204,1.826962
1190,1562,1.740655
1191,355,1.689187
1192,193,1.484383


In [72]:
discounted_recommendations_for_u2_compare = discounted_recommendations_for_u2_df.merge(recommendations_for_u2_df, on='MovieID', how='inner')
discounted_recommendations_for_u2_compare

,MovieID,DiscountedPredictedRating,PredictedRating
0,923,4.706073,3.40
1,919,4.653737,4.10
2,858,4.604465,4.35
3,1213,4.599699,4.40
4,1203,4.555620,4.35
...,...,...,...
1189,204,1.826962,2.95
1190,1562,1.740655,2.80
1191,355,1.689187,2.85
1192,193,1.484383,2.50


In [73]:
recommendations_for_u3_df = pd.DataFrame(recommendations_for_u3, columns=['MovieID', 'PredictedRating'])
recommendations_for_u3_df

,MovieID,PredictedRating
0,171763,4.65
1,912,4.55
2,1537,4.55
3,91542,4.55
4,318,4.50
...,...,...
1189,4015,2.10
1190,455,2.05
1191,673,1.95
1192,1499,1.65


In [74]:
discounted_recommendations_for_u3_df = pd.DataFrame(discounted_recommendations_for_u3, columns=['MovieID', 'DiscountedPredictedRating'])
discounted_recommendations_for_u3_df

,MovieID,DiscountedPredictedRating
0,1214,4.700050
1,2019,4.700013
2,858,4.699950
3,1258,4.650069
4,1203,4.649944
...,...,...
1186,6503,1.850060
1187,1882,1.800169
1188,63992,1.699640
1189,1562,1.650457


In [75]:
discounted_recommendations_for_u3_compare = discounted_recommendations_for_u3_df.merge(recommendations_for_u3_df, on='MovieID', how='inner')   
discounted_recommendations_for_u3_compare

,MovieID,DiscountedPredictedRating,PredictedRating
0,1214,4.700050,3.85
1,2019,4.700013,4.20
2,858,4.699950,4.35
3,1258,4.650069,4.05
4,1203,4.649944,4.35
...,...,...,...
1165,6503,1.850060,2.40
1166,1882,1.800169,2.70
1167,63992,1.699640,2.45
1168,1562,1.650457,2.80


##### Comments
my comments are: nearly all the predicted values decreased after discounted, some of them were slightly decreased and most of them were significantly decreased, and that's because -as i said in task 7- the high bias of the raw cosine similarity that lead to high discounting for the values, and as the not discounted raw cosine similarity lead to give a significant and over estimated values for the prediction and the similarity, the discounted cosine similarity decrease this bias in raw cosine similarity 

#### 9- Find users who get a perfect 1.0 cosine similarity however they have different number of rated items. Discuss why this happens?


In [76]:
def find_perfect_matches(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return

    u_idx = matrix_user_ids.index(uid)
    
    sims = Drs.compute_similarities(u_idx) 
    
    perfect_indices = [i for i, s in enumerate(sims) if s > 0.9999999 and i != u_idx]
    
    target_vec = rating_matrix_np[u_idx]
    target_count = np.count_nonzero(~np.isnan(target_vec))
    
    print(f"\nTarget User: {uid}")
    print(f"Rated Items Count: {target_count}")
    
    if not perfect_indices:
        print("  No perfect (1.0) matches found.")
    else:
        print(f"  Found {len(perfect_indices)} perfect matches:")
        print(f"  {'Neighbor ID':<15} | {'Sim Score':<10} | {'Rated Count':<12}")
        print("  " + "-"*45)
        
        for idx in perfect_indices:
            neighbor_id = matrix_user_ids[idx]
            neighbor_vec = rating_matrix_np[idx]
            neighbor_count = np.count_nonzero(~np.isnan(neighbor_vec))
            
            score = sims[idx]
            
            print(f"  {neighbor_id:<15} | {score:.7f}  | {neighbor_count:<12}")


In [77]:
find_perfect_matches(target_users_ids[0])


Target User: 79060
Rated Items Count: 20
  Found 18088 perfect matches:
  Neighbor ID     | Sim Score  | Rated Count 
  ---------------------------------------------
  16              | 1.0000000  | 175         
  43              | 1.0000000  | 109         
  44              | 1.0000000  | 86          
  46              | 1.0000000  | 129         
  56              | 1.0000000  | 51          
  72              | 1.0000000  | 64          
  75              | 1.0000000  | 55          
  76              | 1.0000000  | 331         
  81              | 1.0000000  | 64          
  89              | 1.0000000  | 54          
  90              | 1.0000000  | 62          
  106             | 1.0000000  | 105         
  107             | 1.0000000  | 151         
  117             | 1.0000000  | 57          
  119             | 1.0000000  | 71          
  124             | 1.0000000  | 77          
  127             | 1.0000000  | 131         
  146             | 1.0000000  | 94          
  159

In [78]:
find_perfect_matches(target_users_ids[1])


Target User: 183483
Rated Items Count: 22
  Found 17568 perfect matches:
  Neighbor ID     | Sim Score  | Rated Count 
  ---------------------------------------------
  15              | 1.0000000  | 62          
  25              | 1.0000000  | 65          
  33              | 1.0000000  | 212         
  44              | 1.0000000  | 86          
  50              | 1.0000000  | 89          
  59              | 1.0000000  | 218         
  74              | 1.0000000  | 132         
  81              | 1.0000000  | 64          
  109             | 1.0000000  | 393         
  110             | 1.0000000  | 86          
  115             | 1.0000000  | 314         
  126             | 1.0000000  | 65          
  137             | 1.0000000  | 117         
  141             | 1.0000000  | 56          
  158             | 1.0000000  | 321         
  174             | 1.0000000  | 170         
  177             | 1.0000000  | 69          
  190             | 1.0000000  | 53          
  19

In [79]:
find_perfect_matches(target_users_ids[2])


Target User: 55243
Rated Items Count: 25
  Found 3942 perfect matches:
  Neighbor ID     | Sim Score  | Rated Count 
  ---------------------------------------------
  66              | 1.0000000  | 98          
  88              | 1.0000000  | 159         
  124             | 1.0000000  | 77          
  126             | 1.0000000  | 65          
  137             | 1.0000000  | 117         
  159             | 1.0000000  | 53          
  164             | 1.0000000  | 58          
  172             | 1.0000000  | 85          
  282             | 1.0000000  | 58          
  312             | 1.0000000  | 65          
  335             | 1.0000000  | 163         
  342             | 1.0000000  | 51          
  345             | 1.0000000  | 69          
  387             | 1.0000000  | 119         
  388             | 1.0000000  | 51          
  435             | 1.0000000  | 58          
  549             | 1.0000000  | 71          
  569             | 1.0000000  | 115         
  623 

##### Dissuction
i guess that's because the bias of the raw cosine, the raw cosine has a high bias that can make it over estimate the the similrity between two users even though they have a few overlaps

#### 10- Find users who rated the common items with any of the target users, can we trust them more than who rated more items than the common items?


In [80]:
def analyze_rater_trust(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return

    u_idx = matrix_user_ids.index(uid)
    u_vec = rating_matrix_np[u_idx]
    u_items = set(np.where(~np.isnan(u_vec))[0])
    
    sims = rs.compute_similarities(u_idx)
    
    only_common = []   # Users who rated ONLY the common items
    all_ = [] # Users who rated common + others
    
    sorted_indices = np.argsort(sims)[::-1]
    
    for v_idx in sorted_indices:
        if v_idx == u_idx: continue
        score = sims[v_idx]
        if score <= 0: break 
        
        v_vec = rating_matrix_np[v_idx]
        v_items = set(np.where(~np.isnan(v_vec))[0])
        
        common = u_items.intersection(v_items)
        if not common: continue
            
        neighbor_id = matrix_user_ids[v_idx]
        
        if len(v_items) == len(common):
            only_common.append((neighbor_id, score, len(common)))
        elif len(v_items) > len(common):
            all_.append((neighbor_id, score, len(common), len(v_items)))

    print(f"\nTarget User: {uid}")
    print(f"Target has rated {len(u_items)} items.")
    print("-" * 60)
    
    print(f"1. SUBSET RATERS (Rated ONLY the common items): Found {len(only_common)}")
    if only_common:
        print(f"   {'User ID':<12} | {'Sim':<8} | {'Common/Total':<15}")
        for nid, score, count in only_common[:5]:
            print(f"   {nid:<12} | {score:.4f}   | {count}/{count}")
            
    print(f"\n2. SUPERSET RATERS (Rated common + others): Found {len(all_)}")
    if all_:
        print(f"   {'User ID':<12} | {'Sim':<8} | {'Common':<8} | {'Total':<8}")
        for nid, score, com, tot in all_[:5]:
            print(f"   {nid:<12} | {score:.4f}   | {com:<8} | {tot:<8}")
            

In [81]:
analyze_rater_trust(target_users_ids[0])


Target User: 79060
Target has rated 20 items.
------------------------------------------------------------
1. SUBSET RATERS (Rated ONLY the common items): Found 0

2. SUPERSET RATERS (Rated common + others): Found 96638
   User ID      | Sim      | Common   | Total   
   40325        | 1.0000   | 2        | 316     
   193016       | 1.0000   | 2        | 54      
   199694       | 1.0000   | 4        | 59      
   41349        | 1.0000   | 2        | 51      
   124456       | 1.0000   | 2        | 113     


In [82]:
analyze_rater_trust(target_users_ids[1])


Target User: 183483
Target has rated 22 items.
------------------------------------------------------------
1. SUBSET RATERS (Rated ONLY the common items): Found 0

2. SUPERSET RATERS (Rated common + others): Found 55149
   User ID      | Sim      | Common   | Total   
   134345       | 1.0000   | 2        | 450     
   46073        | 1.0000   | 2        | 166     
   199627       | 1.0000   | 2        | 516     
   174575       | 1.0000   | 2        | 827     
   90926        | 1.0000   | 2        | 434     


In [83]:
analyze_rater_trust(target_users_ids[2])


Target User: 55243
Target has rated 25 items.
------------------------------------------------------------
1. SUBSET RATERS (Rated ONLY the common items): Found 0

2. SUPERSET RATERS (Rated common + others): Found 108398
   User ID      | Sim      | Common   | Total   
   130341       | 1.0000   | 3        | 113     
   176692       | 1.0000   | 2        | 167     
   200679       | 1.0000   | 2        | 79      
   167535       | 1.0000   | 2        | 59      
   104687       | 1.0000   | 2        | 57      


##### Discussion
No, we usually trust the users who rated more items than those who rated only the common items, even if those who rated only the common items have a "perfect" similarity score

#### 11- What happens if some users ratings are far below the average, however the Cosine is still high; does that mean they agree with others?


In [84]:
# i will write answers here and them move them later to the report

No. High cosine similarity only means they share the same rating pattern (both rate an Item higher than the other Item), not the same values. One user might be "generous" (rating 4 & 5) and the other "harsh" (rating 1 & 2), but if their vectors point in the same direction, they will get a high similarity even their magnitude values,because Cosine ignores the magnitude difference.

#### 12- Should a high cosine similarity always mean strong agreement?


No, because the raw cosine similarity is usually biased and overestimated, so it can't be trusted when it give a high similarity

#### 13- Give your comments in a separate section in your report.

- Raw Cosine is so Biased: It overestimates similarity for users with very few common items (Subset Raters), often giving misleading "perfect" scores.
- Discounting is very importnat: Applying the Discount Factor can successfully penalizing these high similarity, resulting in more reliable (and lower) similarity scores.
- Trust users with more ratings: Users who have rated more items are better neighbors than those who only rated the common items, as they provide new items for recommendation.

### Case study 2:

In [85]:
from MeanCenteredSimilarityRS import MeanCentered_Cosine_similarity_RS

In [86]:
mc_rs = MeanCentered_Cosine_similarity_RS(rating_matrix_np)
mc_rs.fit()

Model fitted. Means & Centered Matrix computed for 110003 users.


#### 1- Repeat the process in Case Study 1 but this time, compute mean-centering


In [87]:
def print_mc_neighbors(uid):
    if uid not in matrix_user_ids:
        print(f"User {uid} missing.")
        return
    
    u_idx = matrix_user_ids.index(uid)
    
    sims = mc_rs.compute_similarities(u_idx)
    
    sorted_idx = np.argsort(sims)[::-1]
    
    print(f"\nTarget User: {uid} (Mean Rating: {mc_rs.means[u_idx]:.2f})")
    print(f"{'Rank':<5} | {'Neighbor ID':<15} | {'MC Sim':<12}")
    print("-" * 40)
    
    count = 0
    for idx in sorted_idx:
        score = sims[idx]
        if score <= 0: continue
            
        nid = matrix_user_ids[idx]
        print(f"{count+1:<5} | {nid:<15} | {score:.6f}")
        
        count += 1
        if count >= 10: break
    
    if count == 0:
        print("  No positively correlated neighbors found.")

In [88]:
print_mc_neighbors(target_users_ids[0])


Target User: 79060 (Mean Rating: 3.65)
Rank  | Neighbor ID     | MC Sim      
----------------------------------------
1     | 85977           | 0.495399
2     | 130623          | 0.395133
3     | 198533          | 0.375870
4     | 64392           | 0.375607
5     | 117411          | 0.364842
6     | 67879           | 0.364739
7     | 56966           | 0.363443
8     | 76632           | 0.346501
9     | 175761          | 0.346292
10    | 132270          | 0.344540


In [89]:
print_mc_neighbors(target_users_ids[1])


Target User: 183483 (Mean Rating: 4.32)
Rank  | Neighbor ID     | MC Sim      
----------------------------------------
1     | 165015          | 0.257510
2     | 196990          | 0.244393
3     | 23254           | 0.238671
4     | 135116          | 0.237827
5     | 74171           | 0.232566
6     | 115611          | 0.213840
7     | 25313           | 0.210525
8     | 136399          | 0.208156
9     | 95269           | 0.202988
10    | 97841           | 0.201420


In [90]:
print_mc_neighbors(target_users_ids[2])


Target User: 55243 (Mean Rating: 3.74)
Rank  | Neighbor ID     | MC Sim      
----------------------------------------
1     | 140244          | 0.376343
2     | 173360          | 0.334081
3     | 4689            | 0.326863
4     | 87452           | 0.309808
5     | 156901          | 0.299030
6     | 9361            | 0.291272
7     | 88928           | 0.290842
8     | 72348           | 0.286188
9     | 33569           | 0.285794
10    | 144872          | 0.282882


#### 2- Identify the top 20% most similar users to each target user based on the adjusted similarities.


In [91]:
def get_mc_top_20(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = matrix_user_ids.index(uid)
    
    all_sims = mc_rs.compute_similarities(u_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)
    
    top_20_indices = sorted_indices[:top_20_count]
    

    top_20_users = []
    for idx in top_20_indices:
        score = all_sims[idx]
        user_id = matrix_user_ids[idx]
        
        if score > 0:
            top_20_users.append((user_id, score))
            
    print(f"\nTarget User: {uid}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Users with Positive Mean-centered Sim in Top 20%: {len(top_20_users)}")
    
    print(f"\n{'- Preview of Top 10 (Mean-centered) -':^40}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Similarity':<10}")
    print("-" * 40)
    
    for rank, (other_uid, score) in enumerate(top_20_users[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.5f}")
        
    print("-" * 40)
    return top_20_users


In [92]:
top_20_mc_u1 = get_mc_top_20(target_users_ids[0])


Target User: 79060
Top 20% Threshold Count: 22000
Users with Positive Mean-centered Sim in Top 20%: 22000

 - Preview of Top 10 (Mean-centered) -  
Rank  | User ID         | Similarity
----------------------------------------
1     | 85977           | 0.49540
2     | 130623          | 0.39513
3     | 198533          | 0.37587
4     | 64392           | 0.37561
5     | 117411          | 0.36484
6     | 67879           | 0.36474
7     | 56966           | 0.36344
8     | 76632           | 0.34650
9     | 175761          | 0.34629
10    | 132270          | 0.34454
----------------------------------------


In [93]:
top_20_mc_u2 = get_mc_top_20(target_users_ids[1])


Target User: 183483
Top 20% Threshold Count: 22000
Users with Positive Mean-centered Sim in Top 20%: 22000

 - Preview of Top 10 (Mean-centered) -  
Rank  | User ID         | Similarity
----------------------------------------
1     | 165015          | 0.25751
2     | 196990          | 0.24439
3     | 23254           | 0.23867
4     | 135116          | 0.23783
5     | 74171           | 0.23257
6     | 115611          | 0.21384
7     | 25313           | 0.21053
8     | 136399          | 0.20816
9     | 95269           | 0.20299
10    | 97841           | 0.20142
----------------------------------------


In [94]:
top_20_mc_u3 = get_mc_top_20(target_users_ids[2])


Target User: 55243
Top 20% Threshold Count: 22000
Users with Positive Mean-centered Sim in Top 20%: 22000

 - Preview of Top 10 (Mean-centered) -  
Rank  | User ID         | Similarity
----------------------------------------
1     | 140244          | 0.37634
2     | 173360          | 0.33408
3     | 4689            | 0.32686
4     | 87452           | 0.30981
5     | 156901          | 0.29903
6     | 9361            | 0.29127
7     | 88928           | 0.29084
8     | 72348           | 0.28619
9     | 33569           | 0.28579
10    | 144872          | 0.28288
----------------------------------------


#### 3- Use these users to predict the unknown ratings.


In [ ]:
def generate_mc_recommendations(uid, K=10):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} is missing.")
        return []
        
    u_idx = matrix_user_ids.index(uid)
    
    user_ratings = mc_rs.matrix[u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_indices)}")
    print("Calculating MC predictions...")
    
    recommendations = []
    
    for i_idx in unrated_indices:
        try:
            pred_val = mc_rs.pred(u_idx, i_idx, K)
            

            if pred_val > 0:
                item_id = matrix_movie_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Mean-Centered):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'MC Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations


In [96]:
mc_recs_u1 = generate_mc_recommendations(target_users_ids[0])



Target User: 79060
Total Unrated Items: 1196
Calculating MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Mean-Centered):
  Rank  | Movie ID     | MC Prediction  
  ----------------------------------------
  1     | 1237         | 4.5131
  2     | 103182       | 4.4709
  3     | 3435         | 4.4090
  4     | 1247         | 4.3757
  5     | 1358         | 4.3517
------------------------------------------------------------


In [97]:
mc_recs_u2 = generate_mc_recommendations(target_users_ids[1])



Target User: 183483
Total Unrated Items: 1194
Calculating MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Mean-Centered):
  Rank  | Movie ID     | MC Prediction  
  ----------------------------------------
  1     | 364          | 5.0000
  2     | 589          | 5.0000
  3     | 741          | 5.0000
  4     | 750          | 5.0000
  5     | 1172         | 5.0000
------------------------------------------------------------


In [98]:
mc_recs_u3 = generate_mc_recommendations(target_users_ids[2])


Target User: 55243
Total Unrated Items: 1191
Calculating MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Mean-Centered):
  Rank  | Movie ID     | MC Prediction  
  ----------------------------------------
  1     | 898          | 4.5463
  2     | 215          | 4.4574
  3     | 1213         | 4.4403
  4     | 1208         | 4.3981
  5     | 5673         | 4.3856
------------------------------------------------------------


#### 4- Calculate DF and DS using threshold B

In [99]:
from MeanCenteredSimilarityRS import Discounted_MeanCentered_Cosine_similarity_RS

In [100]:
d_mc_rs = Discounted_MeanCentered_Cosine_similarity_RS(rating_matrix_np)
d_mc_rs.fit()

Model fitted. Means computed for 110003 users.


In [101]:
total_items = rating_matrix_np.shape[1] 
beta_percentage = 0.30
beta_threshold = int(np.ceil(total_items * beta_percentage))

print(f"Total Items: {total_items}")
print(f"Beta Threshold (30%): {beta_threshold}")
print("-" * 80)


Total Items: 1216
Beta Threshold (30%): 365
--------------------------------------------------------------------------------


In [102]:

def print_mc_discounted_neighbors(uid):
    if uid not in matrix_user_ids:
        print(f"User {uid} not in the matrix.")
        return
        
    u_idx = matrix_user_ids.index(uid)
    
    # 2. Get Statistics
    raw_sims, counts, dfs, dses = d_mc_rs.get_discounted_stats(u_idx, beta_threshold)
    
    # 3. Sort by RAW Similarity to see the penalty effect
    sorted_indices = np.argsort(raw_sims)[::-1]
    
    print(f"\nTarget User: {uid}")
    print(f"{'Rank':<4} | {'Neighbor ID':<12} | {'Common':<6} | {'DF':<6} | {'MS CS':<10} | {'-->':<3} | {'DS (Disc)':<10}")
    print("-" * 80)
    
    count = 0
    for idx in sorted_indices:
        if raw_sims[idx] <= 0: continue
        
        neighbor_id = matrix_user_ids[idx]
        
        n_raw = raw_sims[idx]
        n_count = int(counts[idx])
        n_df = dfs[idx]
        n_ds = dses[idx]
        
        print(f"{count+1:<4} | {neighbor_id:<12} | {n_count:<6} | {n_df:.4f} | {n_raw:.6f} | --> | {n_ds:.6f}")
        
        count += 1
        if count >= 10: break 
        
    if count == 0:
        print("  No positively correlated neighbors found.")

In [103]:
print_mc_discounted_neighbors(target_users_ids[0])


Target User: 79060
Rank | Neighbor ID  | Common | DF     | MS CS      | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 61637        | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 21704        | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 12589        | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 141755       | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 107655       | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 107532       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 16903        | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 55057        | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 916          | 3      | 0.0082 | 1.000000 | --> | 0.008219
10   | 173147       | 2      | 0.0055 | 1.000000 | --> | 0.005479


In [104]:
print_mc_discounted_neighbors(target_users_ids[1])


Target User: 183483
Rank | Neighbor ID  | Common | DF     | MS CS      | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 104858       | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 121466       | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 175467       | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 132327       | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 70558        | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 123961       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 33111        | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 132321       | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 190526       | 2      | 0.0055 | 1.000000 | --> | 0.005479
10   | 33185        | 2      | 0.0055 | 1.000000 | --> | 0.005479


In [105]:
print_mc_discounted_neighbors(target_users_ids[2])


Target User: 55243
Rank | Neighbor ID  | Common | DF     | MS CS      | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 167608       | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 128690       | 2      | 0.0055 | 1.000000 | --> | 0.005479
3    | 40415        | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 121508       | 2      | 0.0055 | 1.000000 | --> | 0.005479
5    | 193080       | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 168327       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 19067        | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 1112         | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 155277       | 3      | 0.0082 | 1.000000 | --> | 0.008219
10   | 70780        | 2      | 0.0055 | 1.000000 | --> | 0.005479


#### 5- Based on the DS values, identify the top 20% similar users.


In [106]:
total_items = rating_matrix_np.shape[1] 
beta_threshold = int(np.ceil(total_items * 0.30))

In [107]:
def get_mc_top_20_discounted(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = matrix_user_ids.index(uid)

    _, _, _, dses = d_mc_rs.get_discounted_stats(u_idx, beta_threshold)
    
    sorted_indices_ds = np.argsort(dses)[::-1]
    
    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)
    
    top_20_indices_ds = sorted_indices_ds[:top_20_count]
    
    top_20_users_ds = []
    for idx in top_20_indices_ds:
        user_id = matrix_user_ids[idx]
        score_ds = dses[idx]
        
        if score_ds > 0:
            top_20_users_ds.append((user_id, score_ds))
            
    print(f"\nTarget User: {uid}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Users with Positive DS in Top 20%: {len(top_20_users_ds)}")
    
    print(f"\n{'- Preview of Top 10 (DS-Based) -':^50}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Discounted Sim':<15}")
    print("-" * 50)
    
    for rank, (other_uid, score) in enumerate(top_20_users_ds[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.6f}")
        
    print("-" * 50)
    return top_20_users_ds

In [108]:
mc_top_20_ds_u1 = get_mc_top_20_discounted(target_users_ids[0])


Target User: 79060
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

         - Preview of Top 10 (DS-Based) -         
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 170779          | 0.034933
2     | 78665           | 0.034051
3     | 9643            | 0.034027
4     | 174431          | 0.033351
5     | 187850          | 0.033339
6     | 38329           | 0.033023
7     | 171882          | 0.032692
8     | 198515          | 0.032597
9     | 57346           | 0.032325
10    | 123366          | 0.031796
--------------------------------------------------


In [109]:
mc_top_20_ds_u2 = get_mc_top_20_discounted(target_users_ids[1])


Target User: 183483
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

         - Preview of Top 10 (DS-Based) -         
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 123465          | 0.040542
2     | 180647          | 0.040056
3     | 197474          | 0.037438
4     | 138940          | 0.037199
5     | 131785          | 0.036734
6     | 60131           | 0.036662
7     | 169086          | 0.036458
8     | 86889           | 0.036433
9     | 70362           | 0.036322
10    | 7248            | 0.036254
--------------------------------------------------


In [110]:
mc_top_20_ds_u3 = get_mc_top_20_discounted(target_users_ids[2])


Target User: 55243
Top 20% Threshold Count: 22000
Users with Positive DS in Top 20%: 22000

         - Preview of Top 10 (DS-Based) -         
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 105431          | 0.040943
2     | 103505          | 0.039752
3     | 196215          | 0.039580
4     | 17700           | 0.039517
5     | 99818           | 0.039400
6     | 19968           | 0.039338
7     | 89339           | 0.038449
8     | 27038           | 0.037875
9     | 31899           | 0.037288
10    | 115957          | 0.037176
--------------------------------------------------


#### 6-Predict ratings again using these updated similarities.

In [ ]:

def generate_mc_discounted_recommendations(K,uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
        
    u_idx = matrix_user_ids.index(uid)
    
    user_ratings = d_mc_rs.matrix[u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_indices)}")
    print("Calculating Discounted MC predictions...")
    
    recommendations = []
    
    for i_idx in unrated_indices:
        try:
            pred_val = d_mc_rs.pred_discounted(u_idx, i_idx, K, beta_threshold)
            
            if pred_val > 0:
                item_id = matrix_movie_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Discounted Mean-centered):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Disc Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations

In [112]:
K = 10


In [113]:
mc_disc_recs_u1 = generate_mc_discounted_recommendations(K,target_users_ids[0])


Target User: 79060
Total Unrated Items: 1196
Calculating Discounted MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Discounted Mean-centered):
  Rank  | Movie ID     | Disc Prediction
  ----------------------------------------
  1     | 1193         | 4.9472
  2     | 910          | 4.8955
  3     | 923          | 4.8887
  4     | 318          | 4.8842
  5     | 908          | 4.8532
------------------------------------------------------------


In [114]:
mc_disc_recs_u2 = generate_mc_discounted_recommendations(K,target_users_ids[1])


Target User: 183483
Total Unrated Items: 1194
Calculating Discounted MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Discounted Mean-centered):
  Rank  | Movie ID     | Disc Prediction
  ----------------------------------------
  1     | 1            | 5.0000
  2     | 6            | 5.0000
  3     | 16           | 5.0000
  4     | 32           | 5.0000
  5     | 47           | 5.0000
------------------------------------------------------------


In [115]:
mc_disc_recs_u3 = generate_mc_discounted_recommendations(K,target_users_ids[2])


Target User: 55243
Total Unrated Items: 1191
Calculating Discounted MC predictions... (this might take a moment)

  Top 5 Recommended Movies (Discounted Mean-centered):
  Rank  | Movie ID     | Disc Prediction
  ----------------------------------------
  1     | 1208         | 5.0000
  2     | 1258         | 5.0000
  3     | 2019         | 5.0000
  4     | 5618         | 5.0000
  5     | 56782        | 4.8949
------------------------------------------------------------


#### 7-Compare the top-k user lists from steps 2 and 5 and comment on any changes.


In [116]:
top_20_mc_u1

[(85977, 0.49539917100437714),
 (130623, 0.3951326045761324),
 (198533, 0.37587038653575533),
 (64392, 0.375607078488687),
 (117411, 0.3648421415549197),
 (67879, 0.3647389069123639),
 (56966, 0.3634433421346058),
 (76632, 0.3465010355328705),
 (175761, 0.3462923765816639),
 (132270, 0.34454006063067083),
 (165163, 0.34420518855257554),
 (144768, 0.33530130770624567),
 (116816, 0.333437487837277),
 (143409, 0.33236116066676846),
 (114442, 0.33037269051163043),
 (136893, 0.3302097498078495),
 (7737, 0.3257279471066466),
 (51114, 0.3246964292633516),
 (145360, 0.32403367519582854),
 (103782, 0.32076033708252466),
 (65508, 0.3165542524627043),
 (29527, 0.314988410390985),
 (168883, 0.3147782100345501),
 (42257, 0.31430498122767725),
 (189978, 0.313619902863743),
 (80496, 0.3131497656734004),
 (101798, 0.31299212902168067),
 (87449, 0.31214699426557946),
 (101457, 0.31069802040173833),
 (165055, 0.3082328256871833),
 (199020, 0.30718293978274774),
 (84527, 0.3061295554778699),
 (148123, 0.

In [117]:
top_20_mc_u1_df = pd.DataFrame(top_20_mc_u1, columns=['UserID', 'Mean-centeredSimilarity'])
top_20_mc_u1_df

,UserID,Mean-centeredSimilarity
0,85977,0.495399
1,130623,0.395133
2,198533,0.375870
3,64392,0.375607
4,117411,0.364842
...,...,...
21995,21915,0.043685
21996,160287,0.043684
21997,138212,0.043681
21998,19960,0.043681


In [118]:
mc_top_20_ds_u1

[(170779, 0.034932696068865),
 (78665, 0.03405075038740985),
 (9643, 0.03402698628970347),
 (174431, 0.03335096171589424),
 (187850, 0.033339378884699775),
 (38329, 0.03302280547156703),
 (171882, 0.0326917991830435),
 (198515, 0.0325971259437519),
 (57346, 0.03232479698286863),
 (123366, 0.031796052003891),
 (129125, 0.031233012910559015),
 (33639, 0.030891821467118726),
 (170457, 0.030882954018982653),
 (55627, 0.030506568653878553),
 (78562, 0.030368283716497982),
 (37121, 0.030276468282973153),
 (51488, 0.030028605146084082),
 (191317, 0.029929486535732446),
 (67544, 0.029866122659550035),
 (145213, 0.029858837497742898),
 (49066, 0.029855870041606294),
 (198632, 0.02982446158332384),
 (24243, 0.029725678977683564),
 (168163, 0.02966342984424663),
 (58777, 0.02958020834489766),
 (24763, 0.0294659306736295),
 (11609, 0.02929020533931939),
 (188670, 0.029269481728563158),
 (130904, 0.029135729334094588),
 (187226, 0.02904272719476348),
 (100293, 0.028924750794267692),
 (198265, 0.028

In [119]:
mc_top_20_ds_u1_df = pd.DataFrame(mc_top_20_ds_u1, columns=['UserID', 'DiscountedMean-centeredSimilarity'])
mc_top_20_ds_u1_df

,UserID,DiscountedMean-centeredSimilarity
0,170779,0.034933
1,78665,0.034051
2,9643,0.034027
3,174431,0.033351
4,187850,0.033339
...,...,...
21995,7095,0.007078
21996,69935,0.007077
21997,39560,0.007077
21998,169575,0.007077


In [120]:
mc_top_20_u1_compare = top_20_mc_u1_df.merge(mc_top_20_ds_u1_df, on='UserID', how='inner')
mc_top_20_u1_compare

,UserID,Mean-centeredSimilarity,DiscountedMean-centeredSimilarity
0,130623,0.395133,0.008166
1,64392,0.375607,0.017676
2,117411,0.364842,0.013997
3,67879,0.364739,0.018584
4,56966,0.363443,0.016699
...,...,...,...
15894,121698,0.043696,0.009616
15895,120688,0.043695,0.007979
15896,13704,0.043686,0.010674
15897,21915,0.043685,0.008973


In [121]:
top_20_mc_u2_df = pd.DataFrame(top_20_mc_u2, columns=['UserID', 'Mean-centeredSimilarity'])   
top_20_mc_u2_df

,UserID,Mean-centeredSimilarity
0,165015,0.257510
1,196990,0.244393
2,23254,0.238671
3,135116,0.237827
4,74171,0.232566
...,...,...
21995,125820,0.016605
21996,14356,0.016604
21997,123441,0.016604
21998,128228,0.016602


In [122]:
mc_top_20_ds_u2_df = pd.DataFrame(mc_top_20_ds_u2, columns=['UserID', 'DiscountedMean-centeredSimilarity'])
mc_top_20_ds_u2_df

,UserID,DiscountedMean-centeredSimilarity
0,123465,0.040542
1,180647,0.040056
2,197474,0.037438
3,138940,0.037199
4,131785,0.036734
...,...,...
21995,11691,0.002972
21996,179822,0.002972
21997,58668,0.002972
21998,188632,0.002971


In [123]:
mc_top_20_u2_compare = top_20_mc_u2_df.merge(mc_top_20_ds_u2_df, on='UserID', how='inner')
mc_top_20_u2_compare

,UserID,Mean-centeredSimilarity,DiscountedMean-centeredSimilarity
0,165015,0.257510,0.007342
1,23254,0.238671,0.014216
2,135116,0.237827,0.031902
3,74171,0.232566,0.011219
4,115611,0.213840,0.014899
...,...,...,...
18719,125820,0.016605,0.003610
18720,14356,0.016604,0.005641
18721,123441,0.016604,0.006308
18722,128228,0.016602,0.005323


In [124]:
top_20_mc_u3_df = pd.DataFrame(top_20_mc_u3, columns=['UserID', 'Mean-centeredSimilarity'])
top_20_mc_u3_df

,UserID,Mean-centeredSimilarity
0,140244,0.376343
1,173360,0.334081
2,4689,0.326863
3,87452,0.309808
4,156901,0.299030
...,...,...
21995,171868,0.060285
21996,6119,0.060285
21997,148610,0.060284
21998,145225,0.060284


In [125]:
mc_top_20_ds_u3_df = pd.DataFrame(mc_top_20_ds_u3, columns=['UserID', 'DiscountedMean-centeredSimilarity'])
mc_top_20_ds_u3_df

,UserID,DiscountedMean-centeredSimilarity
0,105431,0.040943
1,103505,0.039752
2,196215,0.039580
3,17700,0.039517
4,99818,0.039400
...,...,...
21995,6508,0.010921
21996,163141,0.010921
21997,199956,0.010921
21998,191144,0.010920


In [126]:
mc_top_20_u3_compare = top_20_mc_u3_df.merge(mc_top_20_ds_u3_df, on='UserID', how='inner')
mc_top_20_u3_compare

,UserID,Mean-centeredSimilarity,DiscountedMean-centeredSimilarity
0,173360,0.334081,0.029030
1,4689,0.326863,0.027113
2,87452,0.309808,0.024719
3,156901,0.299030,0.027743
4,9361,0.291272,0.026583
...,...,...,...
15422,31337,0.060294,0.018780
15423,19255,0.060291,0.012470
15424,57747,0.060286,0.012319
15425,148610,0.060284,0.015697


##### comments
my observation is the similarities decreased because of the discounting factor showing that even the mean centered cosine similarity can be significantly effected by the bias of the few overlaps, but even though its bias, it's still reliable than the raw cosine similarity without it

#### 8- Compare predictions from steps 3 and 6 and discuss the differences.


In [127]:
mc_recs_u1

[(1237, 4.513063237400311),
 (103182, 4.470864882621541),
 (3435, 4.40901333265355),
 (1247, 4.375707782496337),
 (1358, 4.351721358267951),
 (1225, 4.329421356748922),
 (1213, 4.32768216676774),
 (8638, 4.325992847558762),
 (1221, 4.323086601250233),
 (903, 4.322312763199795),
 (1304, 4.322263581888293),
 (2186, 4.310743519401419),
 (1663, 4.305780352523589),
 (2947, 4.2903754606361195),
 (7099, 4.290061400924834),
 (6350, 4.279702798790974),
 (58, 4.277212197386901),
 (4011, 4.2693725158195),
 (750, 4.267668835006594),
 (2231, 4.246395501225047),
 (1204, 4.237102818852151),
 (912, 4.223659805453982),
 (33166, 4.214044945386819),
 (44555, 4.20884866978291),
 (1203, 4.20882392148451),
 (2997, 4.20605720370194),
 (97304, 4.194483361401458),
 (345, 4.1891801759250615),
 (215, 4.185892451692367),
 (3504, 4.183726595175739),
 (1245, 4.18334268141263),
 (1704, 4.182232633837847),
 (4641, 4.182169606583921),
 (1748, 4.172309891999568),
 (318, 4.171574088720417),
 (5995, 4.1698180155588025),


In [128]:
mc_recs_u1_df = pd.DataFrame(mc_recs_u1, columns=['MovieID', 'MC_PredictedRating'])
mc_recs_u1_df

,MovieID,MC_PredictedRating
0,1237,4.513063
1,103182,4.470865
2,3435,4.409013
3,1247,4.375708
4,1358,4.351721
...,...,...
1191,1562,2.456924
1192,6383,2.429601
1193,355,2.400984
1194,2701,2.314253


In [129]:
mc_disc_recs_u1_df = pd.DataFrame(mc_disc_recs_u1, columns=['MovieID', 'Discounted_MC_PredictedRating'])
mc_disc_recs_u1_df

,MovieID,Discounted_MC_PredictedRating
0,1193,4.947155
1,910,4.895548
2,923,4.888746
3,318,4.884183
4,908,4.853216
...,...,...
1191,173,1.927960
1192,2701,1.875501
1193,3988,1.871584
1194,355,1.798093


In [130]:
mc_recs_u1_compare = mc_recs_u1_df.merge(mc_disc_recs_u1_df, on='MovieID', how='inner')
mc_recs_u1_compare

,MovieID,MC_PredictedRating,Discounted_MC_PredictedRating
0,1237,4.513063,4.351785
1,103182,4.470865,4.443559
2,3435,4.409013,4.760707
3,1247,4.375708,4.395732
4,1358,4.351721,4.218384
...,...,...,...
1191,1562,2.456924,2.428183
1192,6383,2.429601,2.192092
1193,355,2.400984,1.798093
1194,2701,2.314253,1.875501


In [131]:
mc_recs_u2_df = pd.DataFrame(mc_recs_u2, columns=['MovieID', 'MC_PredictedRating'])
mc_recs_u2_df

,MovieID,MC_PredictedRating
0,364,5.000000
1,589,5.000000
2,741,5.000000
3,750,5.000000
4,1172,5.000000
...,...,...
1189,72378,2.706919
1190,6157,2.596007
1191,355,2.568835
1192,34150,2.495985


In [132]:
mc_disc_recs_u2_df = pd.DataFrame(mc_disc_recs_u2, columns=['MovieID', 'Discounted_MC_PredictedRating'])
mc_disc_recs_u2_df

,MovieID,Discounted_MC_PredictedRating
0,1,5.000000
1,6,5.000000
2,16,5.000000
3,32,5.000000
4,47,5.000000
...,...,...
1189,34150,2.606812
1190,6157,2.573609
1191,1562,2.409944
1192,203146,2.341908


In [133]:
mc_recs_u2_compare = mc_recs_u2_df.merge(mc_disc_recs_u2_df, on='MovieID', how='inner')
mc_recs_u2_compare

,MovieID,MC_PredictedRating,Discounted_MC_PredictedRating
0,364,5.000000,4.966417
1,589,5.000000,5.000000
2,741,5.000000,4.630296
3,750,5.000000,5.000000
4,1172,5.000000,5.000000
...,...,...,...
1189,72378,2.706919,2.783336
1190,6157,2.596007,2.573609
1191,355,2.568835,2.742745
1192,34150,2.495985,2.606812


In [134]:
mc_recs_u3_df = pd.DataFrame(mc_recs_u3, columns=['MovieID', 'MC_PredictedRating'])
mc_recs_u3_df

,MovieID,MC_PredictedRating
0,898,4.546304
1,215,4.457367
2,1213,4.440293
3,1208,4.398120
4,5673,4.385579
...,...,...
1186,455,2.305527
1187,2701,2.251867
1188,63992,2.175139
1189,6503,2.137488


In [135]:
mc_disc_recs_u3_df = pd.DataFrame(mc_disc_recs_u3, columns=['MovieID', 'Discounted_MC_PredictedRating'])
mc_disc_recs_u3_df

,MovieID,Discounted_MC_PredictedRating
0,1208,5.000000
1,1258,5.000000
2,2019,5.000000
3,5618,5.000000
4,56782,4.894905
...,...,...
1186,355,2.171213
1187,6503,2.168841
1188,1499,2.080630
1189,1562,2.050727


In [136]:
mc_recs_u3_compare = mc_recs_u3_df.merge(mc_disc_recs_u3_df, on='MovieID', how='inner')
mc_recs_u3_compare

,MovieID,MC_PredictedRating,Discounted_MC_PredictedRating
0,898,4.546304,4.087743
1,215,4.457367,4.254028
2,1213,4.440293,4.527713
3,1208,4.398120,5.000000
4,5673,4.385579,3.841178
...,...,...,...
1186,455,2.305527,2.942134
1187,2701,2.251867,2.314047
1188,63992,2.175139,2.415113
1189,6503,2.137488,2.168841


##### Discussion
- Before Discounting (Task 3): The standard Mean-Centered algorithm found similarities with 1.0 (perfect) who shared very few items. These "noisy" neighbors often have significant ratings, makinf the weighted average prediction higher artificially.
- After Discounting (Task 6): The Discount Factor ($\frac{min(|I_{uv}|, \beta)}{\beta}$) heavily penalized these neighbors because their overlap count ($|I_{uv}|$) was small. Their influence on the prediction formula dropped significantly.

#### 9- Find the users whose mean-centered similarity became -1.0 instead of highly +ve similarity in raw Cosine. Does that make the users who highly +ve raw and men-centered Cosine similarities more reliable?


In [ ]:
def find_flipped_neighbors(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return

    u_idx = matrix_user_ids.index(uid)
    
    raw_sims = rs.compute_similarities(u_idx)
    mc_sims = mc_rs.compute_similarities(u_idx)
 
    flipped_indices = []
    
    for v_idx in range(len(raw_sims)):
        if u_idx == v_idx: continue
        
        r_score = raw_sims[v_idx]
        m_score = mc_sims[v_idx]
        
        if r_score > 0.8 and m_score < -0.5:
            flipped_indices.append((v_idx, r_score, m_score))
            
    print(f"\nTarget User: {uid}")
    
    if not flipped_indices:
        print("  No extreme flips found (High Raw -> Negative MC).")
    else:
        print(f"  Found {len(flipped_indices)} neighbors with flipped similarity:")
        print(f"  {'Neighbor ID':<12} | {'Raw Sim':<10} | {'MC Sim':<10} | {'Status':<10}")
        print("  " + "-" * 50)
        
        flipped_indices.sort(key=lambda x: x[2])
        
        for idx, r_s, m_s in flipped_indices[:10]: 
            nid = matrix_user_ids[idx]
            print(f"  {nid:<12} | {r_s:.4f}     | {m_s:.4f}     | Flipped")

In [142]:
find_flipped_neighbors(target_users_ids[0])


Target User: 79060
  No extreme flips found (High Raw -> Negative MC).


In [143]:
find_flipped_neighbors(target_users_ids[1])


Target User: 183483
  No extreme flips found (High Raw -> Negative MC).


In [144]:
find_flipped_neighbors(target_users_ids[2])


Target User: 55243
  No extreme flips found (High Raw -> Negative MC).


##### Observation
In our specific dataset/targets, extreme flips weren't found. However, mathematically, this occurs when two users have opposite preferences relative to their own means, but their absolute ratings remain in the positive range (for example user A rates items [5, 3] and user B rates [3, 5]) Raw cosine sees vectors pointing in the same quadrant (High Similarity ~0.88), while Mean-Centered sees opposite patterns (Similarity -1.0)

#### 10- If one some users rate only their favorite items, and other users rate a full list of items, do they appear unfairly close or far?

Answer: they will be far, Mean-Centered similarity requires variance to establish a pattern (knowing what a user likes and dislikes)

####  11- Give your comments in a separate section in your report.

- Removed User Bias: Mean-Centering successfully solved the "Scale Problem" (Task 11 of Case Study 1). It differentiated between "generous" raters and "harsh" raters, only finding similarity if their patterns matched, not just their direction

- Still Needs Discounting: Despite these improvements, Mean-Centered similarity is still weak to the "few overlaps" (Task 7). Neighbors with tiny overlaps could still generate perfect 1.0 correlations by luck, proving that Discounting (DS) is essential

### Case Study 3

In [79]:
from Pearson_Sim import Pearson_Correlation_RS

In [80]:
pcc_rs = Pearson_Correlation_RS(rating_matrix_np)
pcc_rs.fit()

Pearson Model fitted. Global means computed for 110003 users.


#### 1- Use Pearson Correlation Coefficient (PCC) to compute similarity between each target user and others.


In [81]:
def print_pearson_neighbors(uid):
    if uid not in matrix_user_ids:
        print(f"User {uid} missing.")
        return
    
    u_idx = matrix_user_ids.index(uid)
    
    
    sims = pcc_rs.compute_similarities(u_idx)
    
    sorted_idx = np.argsort(sims)[::-1]
    
    print(f"\nTarget User: {uid}")
    print(f"{'Rank':<5} | {'Neighbor ID':<15} | {'Pearson Sim':<12}")
    print("-" * 40)
    
    count = 0
    for idx in sorted_idx:
        score = sims[idx]
        if score <= 0: continue
            
        nid = matrix_user_ids[idx]
        print(f"{count+1:<5} | {nid:<15} | {score:.6f}")
        
        count += 1
        if count >= 10: break
    
    if count == 0:
        print("  No positively correlated neighbors found.")

In [82]:
print_pearson_neighbors(target_users_ids[0])


Target User: 79060
Rank  | Neighbor ID     | Pearson Sim 
----------------------------------------
1     | 167778          | 1.000000
2     | 116008          | 1.000000
3     | 54601           | 1.000000
4     | 195070          | 1.000000
5     | 148523          | 1.000000
6     | 45391           | 1.000000
7     | 134510          | 1.000000
8     | 109113          | 1.000000
9     | 19686           | 1.000000
10    | 73089           | 1.000000


In [83]:
print_pearson_neighbors(target_users_ids[1])


Target User: 183483
Rank  | Neighbor ID     | Pearson Sim 
----------------------------------------
1     | 151321          | 1.000000
2     | 94622           | 1.000000
3     | 66793           | 1.000000
4     | 142231          | 1.000000
5     | 4720            | 1.000000
6     | 131178          | 1.000000
7     | 47745           | 1.000000
8     | 91258           | 1.000000
9     | 91180           | 1.000000
10    | 179712          | 1.000000


In [84]:
print_pearson_neighbors(target_users_ids[2])


Target User: 55243
Rank  | Neighbor ID     | Pearson Sim 
----------------------------------------
1     | 50038           | 1.000000
2     | 163634          | 1.000000
3     | 22749           | 1.000000
4     | 118681          | 1.000000
5     | 172039          | 1.000000
6     | 43429           | 1.000000
7     | 135896          | 1.000000
8     | 43305           | 1.000000
9     | 32077           | 1.000000
10    | 90037           | 1.000000


#### 2- Determine the top 20% closest users for each target user.


In [85]:
def get_pearson_top_20(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = matrix_user_ids.index(uid)
    
    all_sims = pcc_rs.compute_similarities(u_idx)
    
    sorted_indices = np.argsort(all_sims)[::-1]
    
    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)
    
    top_20_indices = sorted_indices[:top_20_count]
    
    top_20_users = []
    for idx in top_20_indices:
        score = all_sims[idx]
        user_id = matrix_user_ids[idx]
        
        if score > 0:
            top_20_users.append((user_id, score))
            
    print(f"\nTarget User: {uid}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Users with Positive Pearson Sim in Top 20%: {len(top_20_users)}")
    
    print(f"\n{'- Preview of Top 10 (Pearson) -':^40}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Similarity':<10}")
    print("-" * 40)
    
    for rank, (other_uid, score) in enumerate(top_20_users[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.5f}")
        
    print("-" * 40)
    return top_20_users

In [86]:
pearson_top_20_u1 = get_pearson_top_20(target_users_ids[0])


Target User: 79060
Top 20% Threshold Count: 22000
Users with Positive Pearson Sim in Top 20%: 22000

    - Preview of Top 10 (Pearson) -     
Rank  | User ID         | Similarity
----------------------------------------
1     | 167778          | 1.00000
2     | 116008          | 1.00000
3     | 54601           | 1.00000
4     | 195070          | 1.00000
5     | 148523          | 1.00000
6     | 45391           | 1.00000
7     | 134510          | 1.00000
8     | 109113          | 1.00000
9     | 19686           | 1.00000
10    | 73089           | 1.00000
----------------------------------------


In [87]:
pearson_top_20_u2 = get_pearson_top_20(target_users_ids[1])


Target User: 183483
Top 20% Threshold Count: 22000
Users with Positive Pearson Sim in Top 20%: 19053

    - Preview of Top 10 (Pearson) -     
Rank  | User ID         | Similarity
----------------------------------------
1     | 151321          | 1.00000
2     | 94622           | 1.00000
3     | 66793           | 1.00000
4     | 142231          | 1.00000
5     | 4720            | 1.00000
6     | 131178          | 1.00000
7     | 47745           | 1.00000
8     | 91258           | 1.00000
9     | 91180           | 1.00000
10    | 179712          | 1.00000
----------------------------------------


In [88]:
pearson_top_20_u3 = get_pearson_top_20(target_users_ids[2])


Target User: 55243
Top 20% Threshold Count: 22000
Users with Positive Pearson Sim in Top 20%: 22000

    - Preview of Top 10 (Pearson) -     
Rank  | User ID         | Similarity
----------------------------------------
1     | 50038           | 1.00000
2     | 163634          | 1.00000
3     | 22749           | 1.00000
4     | 118681          | 1.00000
5     | 172039          | 1.00000
6     | 43429           | 1.00000
7     | 135896          | 1.00000
8     | 43305           | 1.00000
9     | 32077           | 1.00000
10    | 90037           | 1.00000
----------------------------------------


#### 3- Predict the missing ratings using these selected users.

In [89]:
def generate_pearson_recommendations(uid, K=10):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} is missing.")
        return []
        
    u_idx = matrix_user_ids.index(uid)
    
    user_ratings = pcc_rs.matrix[u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_indices)}")
    print("Calculating Pearson predictions...")
    
    recommendations = []
    
    for i_idx in unrated_indices:
        try:
            
            pred_val = pcc_rs.pred(u_idx, i_idx, K)
            
            if pred_val > 0:
                item_id = matrix_movie_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Pearson):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'PCC Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations


In [90]:
pearson_recs_u1 = generate_pearson_recommendations(target_users_ids[0])



Target User: 79060
Total Unrated Items: 1196
Calculating Pearson predictions...



  Top 5 Recommended Movies (Pearson):
  Rank  | Movie ID     | PCC Prediction 
  ----------------------------------------
  1     | 720          | 4.7550
  2     | 103182       | 4.5952
  3     | 1199         | 4.5305
  4     | 5618         | 4.5287
  5     | 750          | 4.5238
------------------------------------------------------------


In [91]:
pearson_recs_u2 = generate_pearson_recommendations(target_users_ids[1])



Target User: 183483
Total Unrated Items: 1194
Calculating Pearson predictions...

  Top 5 Recommended Movies (Pearson):
  Rank  | Movie ID     | PCC Prediction 
  ----------------------------------------
  1     | 29           | 5.0000
  2     | 720          | 5.0000
  3     | 1148         | 5.0000
  4     | 1212         | 5.0000
  5     | 1221         | 5.0000
------------------------------------------------------------


In [92]:
pearson_recs_u3 = generate_pearson_recommendations(target_users_ids[2])


Target User: 55243
Total Unrated Items: 1191
Calculating Pearson predictions...

  Top 5 Recommended Movies (Pearson):
  Rank  | Movie ID     | PCC Prediction 
  ----------------------------------------
  1     | 318          | 4.9175
  2     | 1148         | 4.8854
  3     | 1200         | 4.8010
  4     | 2288         | 4.7992
  5     | 7153         | 4.7739
------------------------------------------------------------


#### 4-  Is it correct to compute DF and DS considering threshold ß?

In [93]:
from Pearson_Sim import Discounted_Pearson_RS
d_pcc_rs = Discounted_Pearson_RS(rating_matrix_np)
d_pcc_rs.fit()

Pearson Model fitted. Global means computed for 110003 users.


In [94]:
total_items = rating_matrix_np.shape[1]
beta_threshold = int(np.ceil(total_items * 0.30))
print(f"Beta Threshold (30%): {beta_threshold}")

Beta Threshold (30%): 365


In [95]:
def print_discounted_pearson_neighbors(uid):
    u_idx = matrix_user_ids.index(uid)
    
    raw_sims, counts, dfs, dses = d_pcc_rs.get_discounted_stats(u_idx, beta_threshold)
    
    sorted_indices = np.argsort(raw_sims)[::-1]
    
    print(f"\nTarget User: {uid}")
    print(f"{'Rank':<4} | {'Neighbor ID':<12} | {'Common':<6} | {'DF':<6} | {'PCC':<10} | {'-->':<3} | {'DS (Disc)':<10}")
    print("-" * 80)
    
    count = 0
    for idx in sorted_indices:
        if raw_sims[idx] <= 0: continue
        
        nid = matrix_user_ids[idx]
        n_raw = raw_sims[idx]
        n_count = int(counts[idx])
        n_df = dfs[idx]
        n_ds = dses[idx]
        
        print(f"{count+1:<4} | {nid:<12} | {n_count:<6} | {n_df:.4f} | {n_raw:.6f} | --> | {n_ds:.6f}")
        
        count += 1
        if count >= 10: break

In [96]:
print_discounted_pearson_neighbors(target_users_ids[0])


Target User: 79060
Rank | Neighbor ID  | Common | DF     | PCC        | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 167778       | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 116008       | 4      | 0.0110 | 1.000000 | --> | 0.010959
3    | 54601        | 2      | 0.0055 | 1.000000 | --> | 0.005479
4    | 195070       | 3      | 0.0082 | 1.000000 | --> | 0.008219
5    | 148523       | 2      | 0.0055 | 1.000000 | --> | 0.005479
6    | 45391        | 3      | 0.0082 | 1.000000 | --> | 0.008219
7    | 134510       | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 109113       | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 19686        | 3      | 0.0082 | 1.000000 | --> | 0.008219
10   | 73089        | 4      | 0.0110 | 1.000000 | --> | 0.010959


In [97]:
print_discounted_pearson_neighbors(target_users_ids[1])


Target User: 183483
Rank | Neighbor ID  | Common | DF     | PCC        | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 151321       | 5      | 0.0137 | 1.000000 | --> | 0.013699
2    | 94622        | 5      | 0.0137 | 1.000000 | --> | 0.013699
3    | 66793        | 7      | 0.0192 | 1.000000 | --> | 0.019178
4    | 142231       | 5      | 0.0137 | 1.000000 | --> | 0.013699
5    | 4720         | 3      | 0.0082 | 1.000000 | --> | 0.008219
6    | 131178       | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 47745        | 3      | 0.0082 | 1.000000 | --> | 0.008219
8    | 91258        | 3      | 0.0082 | 1.000000 | --> | 0.008219
9    | 91180        | 3      | 0.0082 | 1.000000 | --> | 0.008219
10   | 179712       | 2      | 0.0055 | 1.000000 | --> | 0.005479


In [98]:
print_discounted_pearson_neighbors(target_users_ids[2])


Target User: 55243
Rank | Neighbor ID  | Common | DF     | PCC        | --> | DS (Disc) 
--------------------------------------------------------------------------------
1    | 50038        | 2      | 0.0055 | 1.000000 | --> | 0.005479
2    | 163634       | 3      | 0.0082 | 1.000000 | --> | 0.008219
3    | 22749        | 3      | 0.0082 | 1.000000 | --> | 0.008219
4    | 118681       | 3      | 0.0082 | 1.000000 | --> | 0.008219
5    | 172039       | 3      | 0.0082 | 1.000000 | --> | 0.008219
6    | 43429        | 2      | 0.0055 | 1.000000 | --> | 0.005479
7    | 135896       | 2      | 0.0055 | 1.000000 | --> | 0.005479
8    | 43305        | 2      | 0.0055 | 1.000000 | --> | 0.005479
9    | 32077        | 2      | 0.0055 | 1.000000 | --> | 0.005479
10   | 90037        | 4      | 0.0110 | 1.000000 | --> | 0.010959


##### Answer 
YES, Even though Pearson Correlation handles "user bias", it does not handle significance and since the PCC is very sensitive to the overlaps, Computing the Discount Factor (DF) allows us to apply Significance Weighting. It penalizes these "low-support" correlations, so that high similarity scores are only awarded to neighbors who share a overlap with the target user. This makes the neighbor selection more reliable.

#### 5- If it is correct, identify the top 20% users using the discounted similarities.


In [99]:
total_items = rating_matrix_np.shape[1] 
beta_threshold = int(np.ceil(total_items * 0.30))

def get_pearson_top_20_discounted(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
    
    u_idx = matrix_user_ids.index(uid)
    _, _, _, dses = d_pcc_rs.get_discounted_stats(u_idx, beta_threshold)
    
    sorted_indices_ds = np.argsort(dses)[::-1]
    
    total_users = len(matrix_user_ids)
    top_20_count = int(total_users * 0.20)
    
    top_20_indices_ds = sorted_indices_ds[:top_20_count]
    
    top_20_users_ds = []
    for idx in top_20_indices_ds:
        user_id = matrix_user_ids[idx]
        score_ds = dses[idx]
        
        if score_ds > 0:
            top_20_users_ds.append((user_id, score_ds))
            
    print(f"\nTarget User: {uid}")
    print(f"Top 20% Threshold Count: {top_20_count}")
    print(f"Users with Positive Discounted Pearson in Top 20%: {len(top_20_users_ds)}")
    
    print(f"\n{'- Preview of Top 10 (Discounted PCC) -':^50}")
    print(f"{'Rank':<5} | {'User ID':<15} | {'Discounted Sim':<15}")
    print("-" * 50)
    
    for rank, (other_uid, score) in enumerate(top_20_users_ds[:10]):
        print(f"{rank+1:<5} | {other_uid:<15} | {score:.6f}")
        
    print("-" * 50)
    return top_20_users_ds

In [100]:
pearson_top_20_ds_u1 = get_pearson_top_20_discounted(target_users_ids[0])


Target User: 79060
Top 20% Threshold Count: 22000
Users with Positive Discounted Pearson in Top 20%: 22000

      - Preview of Top 10 (Discounted PCC) -      
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 78665           | 0.036966
2     | 170779          | 0.036599
3     | 134499          | 0.036561
4     | 9643            | 0.036074
5     | 198515          | 0.035590
6     | 174431          | 0.034937
7     | 33639           | 0.033854
8     | 57346           | 0.033780
9     | 38302           | 0.033606
10    | 56986           | 0.033461
--------------------------------------------------


In [101]:
pearson_top_20_ds_u2 = get_pearson_top_20_discounted(target_users_ids[1])


Target User: 183483
Top 20% Threshold Count: 22000
Users with Positive Discounted Pearson in Top 20%: 19053

      - Preview of Top 10 (Discounted PCC) -      
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 123465          | 0.043960
2     | 180647          | 0.040331
3     | 70362           | 0.038924
4     | 60131           | 0.038082
5     | 197474          | 0.037600
6     | 39227           | 0.037255
7     | 117463          | 0.036744
8     | 131785          | 0.036712
9     | 138940          | 0.036670
10    | 58016           | 0.036462
--------------------------------------------------


In [102]:
pearson_top_20_ds_u3 = get_pearson_top_20_discounted(target_users_ids[2])


Target User: 55243
Top 20% Threshold Count: 22000
Users with Positive Discounted Pearson in Top 20%: 22000

      - Preview of Top 10 (Discounted PCC) -      
Rank  | User ID         | Discounted Sim 
--------------------------------------------------
1     | 17700           | 0.048350
2     | 99818           | 0.047508
3     | 179498          | 0.045798
4     | 134711          | 0.044839
5     | 169154          | 0.044534
6     | 105431          | 0.044368
7     | 139829          | 0.044200
8     | 71177           | 0.043900
9     | 171882          | 0.043842
10    | 115589          | 0.043833
--------------------------------------------------


#### 6- Use these for a second round of prediction.


In [103]:
from tqdm import tqdm
def generate_pearson_discounted_recommendations(uid, K=10):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return []
        
    u_idx = matrix_user_ids.index(uid)
    
    user_ratings = d_pcc_rs.matrix[u_idx]
    unrated_indices = np.where(np.isnan(user_ratings))[0]
    
    print(f"\nTarget User: {uid}")
    print(f"Total Unrated Items: {len(unrated_indices)}")
    print("Calculating Discounted Pearson predictions...")
    
    recommendations = []
    
    for i_idx in tqdm(unrated_indices):
        try:
            pred_val = d_pcc_rs.pred_discounted(u_idx, i_idx, K, beta_threshold)
            
            if pred_val > 0:
                item_id = matrix_movie_ids[i_idx]
                recommendations.append((item_id, pred_val))
        except Exception:
            continue
            
    recommendations.sort(key=lambda x: x[1], reverse=True)
    
    if not recommendations:
        print("  No recommendations generated.")
    else:
        print(f"\n  Top 5 Recommended Movies (Discounted PCC):")
        print(f"  {'Rank':<5} | {'Movie ID':<12} | {'Disc Prediction':<15}")
        print("  " + "-" * 40)
        
        for rank, (movie_id, val) in enumerate(recommendations[:5]):
            print(f"  {rank+1:<5} | {movie_id:<12} | {val:.4f}")
            
    print("-" * 60)
    return recommendations



In [104]:
pearson_disc_recs_u1 = generate_pearson_discounted_recommendations(target_users_ids[0])


Target User: 79060
Total Unrated Items: 1196
Calculating Discounted Pearson predictions...


100%|██████████| 1196/1196 [2:05:31<00:00,  6.30s/it] 


  Top 5 Recommended Movies (Discounted PCC):
  Rank  | Movie ID     | Disc Prediction
  ----------------------------------------
  1     | 908          | 4.9562
  2     | 910          | 4.9348
  3     | 593          | 4.9182
  4     | 1945         | 4.9048
  5     | 318          | 4.8637
------------------------------------------------------------


In [ ]:
pearson_disc_recs_u2 = generate_pearson_discounted_recommendations(target_users_ids[1])


Target User: 183483
Total Unrated Items: 1194
Calculating Discounted Pearson predictions...


  1%|          | 7/1194 [00:29<1:25:18,  4.31s/it]

In [ ]:
pearson_disc_recs_u3 = generate_pearson_discounted_recommendations(target_users_ids[2])

#### 7- Compare similarity lists from steps 2 and 5. Comment. 


In [ ]:
pearson_top_20_u1

In [ ]:
pearson_top_20_u1_df = pd.DataFrame(pearson_top_20_u1, columns=['UserID', 'PearsonSimilarity'])
pearson_top_20_u1_df

In [ ]:
pearson_top_20_ds_u1_df = pd.DataFrame(pearson_top_20_ds_u1, columns=['UserID', 'DiscountedPearsonSimilarity'])
pearson_top_20_ds_u1_df

In [ ]:
pearson_top_20_u1_compare = pearson_top_20_u1_df.merge(pearson_top_20_ds_u1_df, on='UserID', how='inner')
pearson_top_20_u1_compare

In [ ]:
pearson_top_20_u2_df = pd.DataFrame(pearson_top_20_u2, columns=['UserID', 'PearsonSimilarity'])
pearson_top_20_u2_df

In [ ]:
pearson_top_20_ds_u2_df = pd.DataFrame(pearson_top_20_ds_u2, columns=['UserID', 'DiscountedPearsonSimilarity'])
pearson_top_20_ds_u2_df

In [ ]:
pearson_top_20_u2_compare = pearson_top_20_u1_df.merge(pearson_top_20_ds_u1_df, on='UserID', how='inner')
pearson_top_20_u2_compare

In [ ]:
pearson_top_20_u3_df = pd.DataFrame(pearson_top_20_u3, columns=['UserID', 'PearsonSimilarity'])
pearson_top_20_u3_df

In [ ]:
pearson_top_20_ds_u3_df = pd.DataFrame(pearson_top_20_ds_u3, columns=['UserID', 'DiscountedPearsonSimilarity'])
pearson_top_20_ds_u3_df

In [ ]:
pearson_top_20_u3_compare = pearson_top_20_u1_df.merge(pearson_top_20_ds_u1_df, on='UserID', how='inner')
pearson_top_20_u3_compare

##### Dissuction

my observation is that the PCC is very sensitive to the low overlaps and its output can not be trustedif wasn't there enough common items between them, the discounted have successfully decearse the impact of the few overlaps variance and made more reliable similarities

#### 8- Compare prediction results from steps 3 and 6. Discuss.


In [ ]:
pearson_recs_u1

In [ ]:
pearson_recs_u1_df = pd.DataFrame(pearson_recs_u1, columns=['MovieID', 'PCC_PredictedRating'])
pearson_recs_u1_df

In [ ]:
pearson_disc_recs_u1_df = pd.DataFrame(pearson_disc_recs_u1, columns=['MovieID', 'Discounted_PCC_PredictedRating'])
pearson_disc_recs_u1_df

In [ ]:
pearson_recs_u1_compare = pearson_recs_u1_df.merge(pearson_disc_recs_u1_df,on='MovieID', how='inner')
pearson_recs_u1_compare

In [ ]:
pearson_recs_u2_df = pd.DataFrame(pearson_recs_u2, columns=['MovieID', 'PCC_PredictedRating'])
pearson_recs_u2_df

In [ ]:
pearson_disc_recs_u2_df = pd.DataFrame(pearson_disc_recs_u2, columns=['MovieID', 'Discounted_PCC_PredictedRating'])
pearson_disc_recs_u2_df

In [ ]:
pearson_recs_u2_compare = pearson_recs_u2_df.merge(pearson_disc_recs_u2_df,on='MovieID', how='inner')
pearson_recs_u2_compare

In [ ]:
pearson_recs_u3_df = pd.DataFrame(pearson_recs_u3, columns=['MovieID', 'PCC_PredictedRating']) 
pearson_recs_u3_df

In [ ]:
pearson_disc_recs_u3_df = pd.DataFrame(pearson_disc_recs_u3, columns=['MovieID', 'Discounted_PCC_PredictedRating'])
pearson_disc_recs_u3_df

In [ ]:
pearson_recs_u3_compare = pearson_recs_u3_df.merge(pearson_disc_recs_u3_df,on='MovieID', how='inner')
pearson_recs_u3_compare

##### Discussion
- Before Discounting: The PCC algorithm found many similarities with 1.0 who shared very few items. which can lead to misleading output
- After Discounting: The Discount Factor had penalized these misleading outputs to more trustworthy values, because their overlap count ($|I_{uv}|$) was small. Their impact on the prediction dropped significantly.

#### 9- Find the users whose Pearson correlation became negative even though their cosine was +ve, show why this happens?


In [ ]:
def find_cosine_pearson_flips(uid):
    if uid not in matrix_user_ids:
        print(f"Target User {uid} missing.")
        return

    u_idx = matrix_user_ids.index(uid)
    raw_sims = rs.compute_similarities(u_idx)
    pearson_sims = pcc_rs.compute_similarities(u_idx)
    
    flipped_indices = []
    
    for v_idx in range(len(raw_sims)):
        if u_idx == v_idx: continue
        
        r_score = raw_sims[v_idx]
        p_score = pearson_sims[v_idx]
        
        if r_score > 0.7 and p_score < -0.2:
            flipped_indices.append((v_idx, r_score, p_score))
            
    print(f"\nTarget User: {uid}")
    
    if not flipped_indices:
        print("  No significant flips found.")
    else:
        print(f"  Found {len(flipped_indices)} discrepancies:")
        print(f"  {'Neighbor ID':<12} | {'Raw Cosine':<12} | {'Pearson':<12} | {'Status':<10}")
        print("  " + "-" * 55)
        
        flipped_indices.sort(key=lambda x: x[2])
        
        for idx, r_s, p_s in flipped_indices[:10]: 
            nid = matrix_user_ids[idx]
            print(f"  {nid:<12} | {r_s:.4f}       | {p_s:.4f}       | Flipped")    

In [ ]:
find_cosine_pearson_flips(target_users_ids[0])

In [ ]:
find_cosine_pearson_flips(target_users_ids[1])

In [ ]:
find_cosine_pearson_flips(target_users_ids[2])

#### 10- Does Pearson give meaningful output when two users rate ≤20% of the items?

##### Answer
No, because PPC is very sensitive to the few overlaps, so with few number of rated items there will be a few number of the common items which will lead to few overlaps and high variance

#### 11- Find users who have very different rating scales; one always generous, one always strict. Discuss what happens in this case?


In [ ]:
def compare_case_studies(mc_list, pearson_list, uid):
    if not mc_list or not pearson_list:
        print(f"Skipping comparison for User {uid} (lists are empty).")
        return

    set_mc = set([u for u, s in mc_list])
    set_pearson = set([u for u, s in pearson_list])
    
    common = set_mc.intersection(set_pearson)
    
    union_len = len(set_mc.union(set_pearson))
    jaccard = len(common) / union_len if union_len > 0 else 0
    
    print(f"\nTarget User: {uid}")
    print(f"Neighbors in Mean-Centered List: {len(set_mc)}")
    print(f"Neighbors in Pearson List:       {len(set_pearson)}")
    print(f"Overlapping Neighbors:           {len(common)}")
    print(f"Overlap Percentage (Jaccard):    {jaccard:.4f}")
    
    only_in_mc = list(set_mc - set_pearson)[:3]
    only_in_pearson = list(set_pearson - set_mc)[:3]
    
    if only_in_mc:
        print(f"  > Unique to Mean-Centered: {only_in_mc}...")
    if only_in_pearson:
        print(f"  > Unique to Pearson: {only_in_pearson}...")


In [ ]:
compare_case_studies(mc_top_20_ds_u1, pearson_top_20_ds_u1, target_users_ids[0])

In [ ]:
compare_case_studies(mc_top_20_ds_u2, pearson_top_20_ds_u2, target_users_ids[1])

In [ ]:
compare_case_studies(mc_top_20_ds_u2, pearson_top_20_ds_u2, target_users_ids[1])

##### Answer 
- MC-CS (Case 2): Uses Mean of all items. If a user is generally grumpy (Mean=2.0) but rates Sci-Fi movies 3.0, MC-CS sees this as "positive preference" (+1.0).
- Pearson (Case 3): mean of common items. If the common items are all Sci-Fi movies, the local mean might be 3.0. In this context, a 3.0 rating is "neutral" (0.0).

there will be a high overlap between them, Pearson is stricter. It uses the average of only common items, so it ignores "genre bias" (if both users only watched Action movies, Pearson centers based on Action average, not all the average). This often leads to a slightly different, more refined ranking than Mean-Centered Cosine.

#### 12- Find cases where Pearson detected opposite patterns compared to the raw Cosine similarity but based on a smaller number of ratings; do we still trust it?


- Pearson is Superior but Sensitive: Pearson is arguably the most accurate similarity measure because it dynamically adjusts the baseline for every pair of users. However, it is the most sensitive to the Subset Problem (small overlap = extreme scores).
- Discounting is Mandatory: Using Pearson without Discounting (DS) is dangerous in sparse datasets because of the "False Perfect" correlations. Discounting stabilized the predictions in Task 6.
- Conservative Predictions: The shift from Raw Cosine $\to$ Mean-Centered $\to$ Discounted Pearson resulted in progressively more conservative and realistic predictions, filtering out noise and luck.

#### 13- Give your comments in a separate section in your report.

- We noticed that Pearson is so sensitive to the "few overlaps Problem." Users with very small overlaps (e.g., 2-3 items) frequently generated extreme correlations (+1.0 or -1.0), making the raw metric unstable on sparse data.
- Applying the Discount Factor (DF) was not an option but a necessity. It successfully filtered out the "False Perfect" correlations caused by low overlap, stabilizing the predictions and making the neighbor list significantly more reliable especially for PPC.

### Final Task for Part 1: Compare and reflect on the outcomes across Case Studies 1, 2, and 3, considering the impact of similarity metrics and bias adjustment.


##### First Case Study (Raw Cosine):

**Method:** Raw cosine measures the angle between rating vectors, regardless of how to calculate it.

**Result:** Raw cosine is extremely sensitive to the user's bias, meaning it considers "harsh" raters (average score of 2.0) to be the same as "generous" raters (average score of 4.5) if their rating vectors have the same direction. It generates non-true similarities if two users agree on a rating but do not agree on the relative order of the other ratings.

**Conclusion:** The results are unreliable for finding analogous preferences due to high sensitivity to user bias. It is good for checking agreement on magnitude ratings but is not good for finding similar patterns of preference.

##### Case Study 2 (Mean-Centered Cosine):

**Mechanism**: Subtracts the user's Mean ($\bar{r}_u$) from all ratings.

**Outcome**: Solved the "Scale Problem." It correctly identified that a 3.0 rating is "bad" for a generous user but "good" for a harsh user. It filtered out the "False Positives" seen in Raw Cosine.

**conclusion**: Much better. It captures the true pattern of preference (likes vs. dislikes) rather than just volume.

##### Case Study 3 (Pearson Correlation):

**Mechanism**: Subtracts the average of only the common items for each pair.

**Outcome**: It removed "Context Bias.", Pearson adjusted the baseline to that specific genre average, revealing if they actually agreed on which Action movies were best. 

**Conclusion**: Mathematically superior for measuring linear correlation, but so weak on small sample sizes.